# Context
* Scientific Goal: Understand if and how emotional valence affects risk-taking
* Data Set: 35 subjects participating in a 90 trial mood manipulation task, divided into a 20 subject train and 15 subject test set
* Statistical Goal: Create a statistical test to detect if mood belongs in the Markov blanket of risk-taking behavior
* Computational Goal: Explore the data sets, build task-specific features, build features that encode competing scientific hypotheses about emotional valence and risk-taking, examine the stability of features over varying regularization and data resampling, build logisitc regressions and neural networks and interpret their coefficients
* Full thesis: https://drive.google.com/file/d/1rFG2a6AN9RBRsbDSq0blgwnVahAHY399/view?usp=sharing 

# Table Of Contents: <a class="anchor" id="toc"></a>
* [Requirements and Imports](#Requirements-and-Imports)
* [Load and Explore Data](#Load)
    * [Per Trial Data](#per-trial)
        * [load data and calculate task-based features](#task-features)
        * [explore](#e1)
    * [Per Subject Data](#per-subj)
        * [create a per subject data frame 'gps'](#gps)
        * [explore](#e2)
* [Model Evaluation Helper Functions](#Evaluating-Models:-Helper-Functions)
* [Subsetting Data Helper Functions](#Subsetting-Data-Helper-Functions)
* [Heatmaps Helper Functions](#Heatmaps-Helper-Functions)
* [Feature Selection](#feat)
    * [Create Normalized Main Features and Evaluate Collinearity](#Create-Normalized-Main-Features-and-Evaluate-Collinearity)
        * [define helper functions](#define-helper-functions-1)
        * [generate and save features](#generate-and-save-features)
        * [evaluate collinearity](#evaluate-collinearity)
        * [visualize standardization](#visualize-standardization)
    * [Calculate Quadratic Features](#Calculate-Quadratic-Features)
    * [General Function to Standardize Features](#General-Function-to-Standardize-Features)
    * [Stability Plots and Regularization Plots](#Stability-Plots-and-Regularization-Plots)
        * [define helper functions](#define-helper-functions-2)
        * [create plots](#create-plots)
    * [Average Stability Calculations](#Average-Stability-Calculations)
        * [save stability values](#save-stability-values)
        * [save indicies of stability values](#save-indicies-of-stability-values)
    * [Picking Number of Most Stable Features](#Picking-Number-of-Most-Stable-Features)
* [Logistic Regression Modeling and Interpretation](#Logistic-Regression-Modeling-and-Interpretation)
    * [Estimate LOOCV accuracy](#Estimate-LOOCV-accuracy-(leaving-out-one-subject's-90-trials))
    * [Estimate Transfer Accuracy](#Estimate-Transfer-Accuracy)
    * [Estimate Weights of a Model Trained on Entire Data Set](#Estimate-Weights-of-a-Model-Trained-on-Entire-Data-Set)
    * [Visualize Weights of a Model Trained on Entire Data Set](#Visualize-Weights-of-a-Model-Trained-on-Entire-Data-Set)
    * [Compare Weights of a Model Trained on Entire Data Set](#Compare-Weights-of-a-Model-Trained-on-Entire-Data-Set)
* [Neural Network Modeling and Interpretation](#Neural-Network-Modeling-and-Interpretation)
    * [Estimate LOOCV accuracy](#Estimate-LOOCV-accuracy)
    * [Generate Hessian Values](#Generate-Hessian-Values)
    * [Visualize Hessian Matrices](#Visualize-Hessian-Matrices)

# Requirements and Imports <a class="anchor" id="Requirements-and-Imports"></a>


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
import sklearn.metrics
from scipy.stats.stats import pearsonr
import math
import csv
from decimal import Decimal
import tensorflow as tf
from tensorflow.python.ops import array_ops
import subprocess
import seaborn as sns
import time

colors = ['aqua', 'darkorange', 'cornflowerblue']

pd.options.display.max_columns = 200

[Back to Table of Contents](#toc)

# Load and Explore Data <a class="anchor" id="Load"></a>
Output: 
* visualizations of per trial and per subject data
* a csv file called 'gps_[experiment].csv' which lists subjects and their characteristics. The subject ids in this file will be used for future analyses that look at cross-subject validation for example

## Per Trial Data <a class="anchor" id="per-trial"></a>
* What is the demographic information of subjects?
* What is the range of all subject characteristics?
* How does the choice to gamble change over time per subject?
* How do attributes of the gambling task change over time?


[Back to Table of Contents](#toc)

### load data and calculate task-based features <a class="anchor" id="task-features"></a>

In [ ]:
data = pd.read_csv("10data_for_jess_random.csv")
ntrials = data.time.max() + 1
nrows = data.shape[0]

data['PredictionError'] = data['Actual'] \
                          - data['Gamble'] * (data['Outcome1Amount'] + data['Outcome2Amount']) / 2 \
                          - (1 - data['Gamble']) * data['CertainAmount']
data['ExpectedGamble'] = (data['Outcome1Amount'] + data['Outcome2Amount']) / 2
data['DiffCertainExpectedGamble'] = data['CertainAmount'] - data['ExpectedGamble']
data['HigherOutcome1'] = data[["Outcome1Amount", "Outcome2Amount"]].max(axis=1)
data['LowerOutcome1'] = data[["Outcome1Amount", "Outcome2Amount"]].min(axis=1)
data['Mood'] = data['Happiness']

# map subject to all row indices
ID = {}
for i in range(nrows):
    id = int(data.iloc[i,0])
    ID[id] = ID.get(id, []) + [i]
nsubj = len(ID)

# map subject characteristics to strings
def ptype2str(x):
    str = ""
    if x == 1:
        str = "Depressed"
    else:
        str = "Healthy"
    return str

def gender2str(x):
    str = ""
    if x == 1:
        str = "Female"
    else:
        str = "Male"
    return str

[Back to Table of Contents](#toc)

### explore <a class="anchor" id="e1"></a>

In [ ]:
# Create a dictionary to store all subjects' trials
subjDF = {}
subjs = ID.keys()
for subj in subjs:
    # subject data frame, string ID, string patient type
    subjD = data.iloc[ID[subj],:]
    subjID = str(subjD.loc[:,'subject_id'].iloc[0])
    subjptype = ptype2str(subjD.loc[:,'ptype'].iloc[0])
    subjDF[subj] = subjD

subjs = ID.keys()
gps = []
for subj in subjs:
    # subject data frame, string ID, string patient type
    subjD = data.iloc[ID[subj],:]
    subjID = str(subjD.loc[:,'subject_id'].iloc[0])
    subjptype = ptype2str(subjD.loc[:,'ptype'].iloc[0])
    gp = sum(subjD.loc[:,'Gamble']) / subjD.loc[:,'Gamble'].size
    gps.append(gp)
hgp = max(gps)
lgp = min(gps)
print (hgp, lgp)

# Visualizing all probabilities
subjs = ID.keys()
c = 0
gps = []
for subj in subjs:
    # subject data frame, string ID, string patient type
    subjD = data.iloc[ID[subj],:]
    subjID = str(subjD.loc[:,'subject_id'].iloc[0])
    subjptype = ptype2str(subjD.loc[:,'ptype'].iloc[0])
    gp = sum(subjD.loc[:,'Gamble']) / subjD.loc[:,'Gamble'].size
    gps.append(gp)
bins = np.linspace(0, 1, 100)
plt.hist(gps)
plt.title("Gambling Probabilities")
plt.xlabel("Gambling Probability")
plt.show()

# Visualizing the Subject Data
subjs = ID.keys()#[0:5] + ID.keys()[25:30]
c = 0
for subj in subjs:
    # subject data frame, string ID, string patient type
    subjD = data.iloc[ID[subj],:]
    subjID = str(subjD.loc[:,'subject_id'].iloc[0])
    subjptype = ptype2str(subjD.loc[:,'ptype'].iloc[0])

    plt.plot('time', 'CertainAmount', data=subjD, label="non gamble amount")
    plt.plot('time', 'Outcome1Amount', data=subjD, label="gamble amount 1")
    plt.plot('time', 'Outcome2Amount', data=subjD, label="gamble amount 2")
    plt.plot('time', 'Actual', data=subjD, label="round outcome amount", marker='o')
    plt.plot('time', 'PredictionError', data=subjD, label="prediction error", marker='o')
    plt.plot('time', 'ExpectedGamble', data=subjD, label="expected gamble amount", marker='o')
    plt.plot('time', 'CertainAmount', data=subjD, label="certain amount", marker="o")
    plt.plot('time', 'DiffCertainExpectedGamble', data=subjD, label="diff", marker="o")
    plt.title(subjID + " " + subjptype)
    plt.legend(loc=3)
    plt.show()

    bins = np.linspace(0, 1, 50)
    plt.hist([x for x in subjD.loc[:,'Happiness'] if str(x) != 'nan'], bins)
    plt.title("Moods of " + subjID + " " + subjptype)
    plt.show()

    bins = np.linspace(-10, 10, 50)
    plt.hist(subjD.loc[:, "CertainAmount"], bins, alpha=0.5, label="CR")
    plt.hist(subjD.loc[:,'HigherOutcome1'], bins, alpha=0.5, label="H")
    plt.hist(subjD.loc[:, 'LowerOutcome1'], bins, alpha=0.5, label="L")
    plt.legend(loc='upper right')
    plt.show()
    c += 1
    print (c)

    h = subjD.loc[:, 'Happiness']
    xs = np.arange(len(h))
    yh = np.array(h)
    yhm = np.isfinite(yh)
    plt.plot(xs[yhm], yh[yhm], label="happiness", marker='o')
    plt.plot('time', 'Gamble', data=subjD, label="decision to gamble", marker='o')
    plt.ylim((-1.5,1.5))
    plt.legend(loc=3)
    plt.show()

[Back to Table of Contents](#toc)

## Per Subject Data <a class="anchor" id="per-subj"></a>
* How does overall gambling probability vary with subject characteristics?
* Can we predict a subject's overall gambling probability from subject characteristics?

[Back to Table of Contents](#toc)

### create a per subject data frame: gps <a class="anchor" id="gps"></a>

In [ ]:
## Set Up Dataframe: gps
# every row has a subject characteristics &
# global gambling probabilities

final_name = "gps_random.csv"

gps = data.drop_duplicates(subset = ['subject_id', 'age', 'gender',
                                     'ptype', 'MFQ', 'SHAPS'])
gps = gps.drop(['time', 'CertainAmount',
                'Outcome1Amount', 'Outcome2Amount',
                'Gamble', 'Actual', 'Happiness'], axis = 1)

gps['GamblingProbability'] = 0.0

# index of 'Gamble'
gi = list(data).index('Gamble')

# index of 'GamblingProbability'
gpi = list(gps).index('GamblingProbability')

# calculate subject overall gambling probability
for i in range(nsubj):
    id = int(gps.iloc[i,0])
    indices = ID[id]
    gambles = data.iloc[indices,gi]
    gps.iloc[i,gpi] = float(sum(gambles))/float(len(gambles))

gps.to_csv(final_name)

[Back to Table of Contents](#toc)

### explore <a class="anchor" id="e2"></a>

In [ ]:
## Data Characteristics get ranges of subject characteristics
shapss = gps.loc[:,'SHAPS']
sl = min(shapss)
sh = max(shapss)
print (sl, sh)

mfqs = gps.loc[:,'MFQ']
ml = min(mfqs)
mh = max(mfqs)
print (ml, mh)

ages = gps.loc[:,'age']
al = min(ages)
ah = max(ages)
print (al, ah)

genders = gps.loc[:,'gender']
counts = genders.value_counts()
f = counts[1] / ntrials
m = counts[2] / ntrials
print (f, m)

dep = gps.loc[:,'ptype']
counts = dep.value_counts()
d = counts[1] / ntrials
h = counts[2] / ntrials
print (d, h)

## Plot data in gps
xnames = ['age', 'gender', 'ptype', 'MFQ', 'SHAPS']
for xname in xnames:
    gps.plot(kind='scatter',x=xname,y='GamblingProbability',color='red')
    plt.show()

## Run linear regressions
y = gps.iloc[:,gpi]
X = gps.loc[:,xnames]

## Linear Regression

feature = 'ptype'
y = gps.iloc[:,gpi]
x = gps.loc[:,feature].values.reshape((-1, 1))
model = LinearRegression()
model.fit(x, y)
r_sq = model.score(x, y)
print('coefficient of determination:', r_sq)
print('intercept:', model.intercept_)
print('slope:', model.coef_)
y_pred = model.intercept_ + model.coef_ * x

plt.plot(x, y, '.')
plt.plot(x, model.intercept_ + model.coef_ * x, '-')
plt.xlabel(feature)
plt.ylabel("Gambling Probability")
plt.title("GP vs. " + feature + ": r^2 = " + str(r_sq))
plt.show()

## Pearson Correlation

feature = 'ptype'
feature2 = 'SHAPS'
#y = gps.iloc[:,gpi]
x1 = gps.loc[:,feature]
x2 = gps.loc[:,feature2]

r = pearsonr(x1,x2)
print (r)

[Back to Table of Contents](#toc)

# Evaluating Models: Helper Functions

In [ ]:
# Evaluate Per Trial Predictive Accuracy


def accthreshold(obsvs, preds, threshold):
    '''
    Evaluate the accuracy of a prediction given a list of probabilitstic predictions (preds), 
    a probabilititic threshold (threshold), and the binary ground truth (obsvs)
    '''
    preds_bool = preds > threshold
    acc = sklearn.metrics.accuracy_score(obsvs, preds_bool)
    return acc


def maxacc(obsvs, preds):
    '''
    Given a list of probabilitstic predictions (preds) and the binary ground truth (obsvs),
    evaluate the largest accuracy achieved across probabilistic thresholds 0.01, 0.02, ... 1.00
    '''
    thresholds = [float(i) / float(100) for i in range(1,100,1)]
    temp = 0
    print ("the accs are ")
    for threshold in thresholds:
        temp2 = accthreshold(obsvs, preds, threshold)
        print (temp2)
        temp = max(temp, temp2)
    return temp

def balancedacc(obsvs, preds, threshold):
    '''
    Evaluate the balanced accuracy of a prediction given a list of probabilitstic predictions (preds),
    a probabilititic threshold (threshold), and the binary ground truth (obsvs)
    '''
    preds_bool = preds > threshold
    conditionpositive = sum(obsvs)
    tn, fp, fn, tp = sklearn.metrics.confusion_matrix(obsvs, preds_bool).ravel()
    tpr = float(tp) / float(tp + fn)
    fpr = float(tn) / float(tn + fp)
    bacc = (tpr + fpr) / 2
    return bacc

def maxbacc(obsvs, preds):
    '''
    Given a list of probabilitstic predictions (preds) and the binary ground truth (obsvs),
    evaluate the largest balanced accuracy achieved across probabilistic thresholds 0.01, 0.02, ... 1.00
    '''
    thresholds = [float(i) / float(100) for i in range(1,100,1)]
    temp = 0
    print ("the accs are ")
    for threshold in thresholds:
        temp2 = balancedacc(obsvs, preds, threshold)
        print (temp2)
        temp = max(temp, temp2)
    return temp


# Evaluate Per Subject Predictive Accuracy

def get_indices(x,xs):
    '''
    Find the indicies of an element x in a list xs
    '''
    return [i for (i,y) in zip(range(len(xs)), xs) if y == x]


def subjauc(subjs, obsvs, preds):
    '''
    Evaluate the AUC of a prediction averaged across subjects,
    given the probabilistic predictions (preds),
    the binary ground truth (obsvs),
    and the subject ID of each trial (subjs)
    e.g.
    obsvs : [1  , 1  , 0  , 1  , 0  , 1  , 0  , 1  , 1]
    preds : [0.8, 0.4, 0.2, 0.8, 0.6, 0.2, 0.6, 0.4, 0.8] 
    subjs : [a, a, a, b, b, b, c, c, c]

    aucs  : [1.0, 0.5, 0.5]
    average AUC : 0.66
    '''
    ss = set(subjs)
    aucs = []
    for s in ss:
        sind = get_indices(s, subjs)
        obsvs_s = obsvs[sind]
        preds_s = preds[sind]
        fpr, tpr, thresholds = sklearn.metrics.roc_curve(obsvs_s, preds_s)
        auc_s = sklearn.metrics.auc(fpr, tpr)
        aucs.append(auc_s)
    return np.nanmean(aucs)





[Back to Table of Contents](#toc)

# Subsetting Data Helper Functions

In [ ]:
# Helper Functions for Subsetting Data by Subject ID or Feature Column Index


def subjXY(sid, x, y):
    '''
    Subset the x and y values corresponding to a given subject sid

    sid = subject ID string
    x = np array of features including all subject ID strings as column 0
    y = np array of targest including all subject ID strings as column 0

    e.g.
    sid = 'a'
    x = np.array([['a', 0, 7, 2], ['b', 1, 4, 5], ['c', 1, 5, 3]])
    y = np.array([['a', 1], ['b', 1], ['c', 0]])
    subjXY(sid,x,y) = [np.array([0, 7, 2]), np.array([1])]
    '''
    sidscol = np.array(x[:,0])
    inds = np.where(sidscol == sid)
    features = np.squeeze(x[inds,1:])
    targets = np.squeeze(y[inds,1:])
    return [features, targets]

def getXY(sids, x, y):
    
    '''
    Subset the x and y values corresponding to a list of subject IDs sids

    sids = a list of subject ID strings
    x = np array of features including all subject ID strings as column 0
    y = np array of targest including all subject ID strings as column 0

    e.g.
    sids = ['a','b']
    x = np.array([['a', 0, 7, 2], ['b', 1, 4, 5], ['c', 1, 5, 3]])
    y = np.array([['a', 1], ['b', 1], ['c', 0]])
    getXY(sids,x,y) = [np.array([0, 7, 2], [1, 4, 5]), np.array([1, 1])]
    '''
    
    xs = np.zeros(shape=[x.shape[0]-1, x.shape[1]-1])
    ys = np.zeros(shape=[y.shape[0]-1, 1])
    xbool = np.zeros(shape=y.shape[0]-1)

    counter = 0
    for i in range(len(sids)):
        [sx, sy] = subjXY(sids[i], x, y)
        size = sx.shape[0]
        xs[range(counter, counter+size), :] = sx
        ys[range(counter, counter+size)] = sy.reshape(len(sy),1)
        counter = counter + size
    return [xs, ys]

def subjY(sid, y):
    '''
    Subset the y values corresponding to a given subject sid

    sid = subject ID string
    y = np array of targest including all subject ID strings as column 0

    e.g.
    sid = 'a'
    y = np.array([['a', 1], ['b', 1], ['c', 0]])
    subjY(sid,y) = np.array([1])
    '''
    sidscol = np.array(y[:,0])
    inds = np.where(sidscol == sid)
    targets = np.squeeze(y[inds,1:])
    return targets

def notsubjXY(sid, x, y):
    '''
    Subset the x and y values not corresponding to a given subject sid

    sid = subject ID string
    x = np array of features including all subject ID strings as column 0
    y = np array of targest including all subject ID strings as column 0

    e.g.
    sid = 'a'
    x = np.array([['a', 0, 7, 2], ['b', 1, 4, 5], ['c', 1, 5, 3]])
    y = np.array([['a', 1], ['b', 1], ['c', 0]])
    notsubjXY(sid,x,y) = [np.array([[1, 4, 5],[1, 5, 3]]), np.array([1,0])]
    '''
    sidscol = np.array(x[:,0])
    inds = np.where(sidscol != sid)
    features = np.squeeze(x[inds,1:])[1:,]
    targets = np.squeeze(y[inds,1:])[1:]
    return [features, targets]

def notsubjY(sid, y):
    '''
    Subset the y values not corresponding to a given subject sid

    sid = subject ID string
    y = np array of targest including all subject ID strings as column 0

    e.g.
    sid = 'a'
    y = np.array([['a', 1], ['b', 1], ['c', 0]])
    notsubjY(sid,y) = [np.array([1,0])]
    '''
    sidscol = np.array(y[:,0])
    inds = np.where(sidscol != sid)
    targets = np.squeeze(y[inds,1:])[1:]
    return targets

def subsetx(x, l):
    '''
    Subset a list of l specific features of x
    x = np array of features including all subject ID strings as column 0
    l = 0 indexed list of desired features

    e.g.
    x = np.array([['a', 0, 7, 2], ['b', 1, 4, 5], ['c', 1, 5, 3]])
    l = [1,3]
    subsetx(x,l) = np.array([['a', 0, 2], ['b', 1, 5], ['c', 1, 3]])
    '''
    xnew = np.zeros(shape=(x.shape[0], len(l) + 1))
    xnew[:,0] = x[:,0]
    for col in range(len(l)):
        colind = l[col]
        xnew[:,col+1] = x[:,colind]
    return xnew

def subsetxfile(filename, feats):
    '''
    Subset a list of l specific features of x
    filename = the filename of a file with the 0th row as the names of features 
               and the 0th column as subject ID strings
    feats = 0-indexed list of desired features

    e.g.
    filename = 'feats.csv' containing {columns = ['subject ID', 'depression status', 'age', 'anhedonia score'], 
                                       data = [['a', 0, 7, 2], ['b', 1, 4, 5], ['c', 1, 5, 3]]}
    feats = [1,3]
    subsetxfile(filename, feats) = np.array([['a', 0, 2], ['b', 1, 5], ['c', 1, 3]])
    '''
    x = np.genfromtxt(filename, delimiter=",")
    xnew = np.zeros(shape=(x.shape[0], len(feats) + 1))
    xnew[:,0] = x[:,0]
    for col in range(len(feats)):
        colind = feats[col]
        xnew[:,col+1] = x[:,colind]
    return xnew


def subsetfilesubj(filename, sids):
    '''
    Subset all the features in filename corresponding to subjects in the list sids
    filename = the filename of a file with the 0th row as the names of features 
               and the 0th column as subject ID strings
    sids = list of desired subject IDs

    e.g.
    filename = 'feats.csv' containing {columns = ['subject ID', 'depression status', 'age', 'anhedonia score'], 
                                       data = [['a', 0, 7, 2], ['b', 1, 4, 5], ['c', 1, 5, 3]]}
    sids = ['a','b']
    subsetfilesubj(filename,sids) = np.array([[0, 7, 2], [1, 4, 5]])
    '''
    x = np.genfromtxt(filename+'.csv', delimiter=",")
    sidscol = np.array(x[:,0])
    inds = []
    for sid in sids:
        ind = np.where(sidscol == sid)
        inds.append(ind)
    inds = np.array(inds).reshape((-1))
    fts = np.squeeze(x[inds,])
    with open(filename + '_overlap_frm'+ traintype + '.csv', 'w') as writeFile:
        writer = csv.writer(writeFile)
        writer.writerow(np.zeros(x.shape[1]))
        writer.writerows(fts)
    writeFile.close()
    return fts[:,1:]

def subsetfilesubjSUBJS(filename, sids):
    '''
    Subset the list of subject ID indicies in filename corresponding to subjects in the list sids
    filename = the filename of a file with the 0th row as the names of features 
               and the 0th column as subject ID strings
    sids = list of desired subject IDs

    e.g.
    filename = 'feats.csv' containing {columns = ['subject ID', 'depression status', 'age', 'anhedonia score', 'trial'], 
                                       data = [['a', 0, 7, 2, 1], ['a', 0, 7, 2, 2],
                                               ['b', 1, 4, 5, 1], ['b', 1, 4, 5, 2],
                                               ['c', 1, 5, 3, 1], ['c', 1, 5, 3, 2]]}
    sids = ['a','b']
    subsetfilesubjSUBJS(filename,sids) = np.array(['a','a','b','b'])
    '''
    x = np.genfromtxt(filename+'.csv', delimiter=",")
    sidscol = np.array(x[:,0])
    inds = []
    for sid in sids:
        ind = np.where(sidscol == sid)
        inds.append(ind)
    inds = np.array(inds).reshape((-1))
    fts = np.squeeze(x[inds,])
    with open(filename + '_overlap_frm'+ traintype + '.csv', 'w') as writeFile:
        writer = csv.writer(writeFile)
        writer.writerow(np.zeros(x.shape[1]))
        writer.writerows(fts)
    writeFile.close()
    return fts[:,0]

def strip(m):
    '''
    input: matrix
    output: matrix without column names or row names
    '''
    m = m[1:, 1:]
    return m

def stripfile(filename):
    '''
    input: filename
    output: matrix without column names or row names
    '''
    m = np.genfromtxt(filename, delimiter=",")
    m = m[1:, 1:]
    return m

def getcolumnnames(filename):
    '''
    Given the filename of a csv file
    return the columnnames
    
    e.g. getcolumnnames("results.csv") = ['model #', 'AUC', 'ACC', 'Subject ACC']
    '''
    with open(filename, "rt") as f:
        reader = csv.reader(f)
        i = next(reader)
    return i

[Back to Table of Contents](#toc)

# Heatmaps Helper Functions

In [ ]:
# Code to Visualize Heatmaps

# https://matplotlib.org/3.1.1/gallery/images_contours_and_fields/image_annotated_heatmap.html

def heatmap(data, row_labels, col_labels, ax=None,
            cbar_kw={}, cbarlabel="", **kwargs):
    '''
    Create a heatmap from a numpy array and two lists of labels.
    [Back to Table of Contents](#toc)Parameters
    ----------
    data
        A 2D numpy array of shape (N, M).
    row_labels
        A list or array of length N with the labels for the rows.
    col_labels
        A list or array of length M with the labels for the columns.
    ax
        A `matplotlib.axes.Axes` instance to which the heatmap is plotted.  If
        not provided, use current axes or create a new one.  Optional.
    cbar_kw
        A dictionary with arguments to `matplotlib.Figure.colorbar`.  Optional.
    cbarlabel
        The label for the colorbar.  Optional.
    **kwargs
        All other arguments are forwarded to `imshow`.
    '''

    if not ax:
        ax = plt.gca()

    # Plot the heatmap
    im = ax.imshow(data, **kwargs)

    # Create colorbar
    cbar = ax.figure.colorbar(im, orientation= "horizontal", ax=ax, **cbar_kw)
    cbar.ax.set_ylabel(cbarlabel, rotation=0, va="bottom")

    # We want to show all ticks...
    ax.set_xticks(np.arange(data.shape[1]))
    ax.set_yticks(np.arange(data.shape[0]))
    # ... and label them with the respective list entries.
    ax.set_xticklabels(col_labels)
    ax.set_yticklabels(row_labels)

    # Let the horizontal axes labeling appear on top.
    ax.tick_params(top=True, bottom=False,
                   labeltop=True, labelbottom=False)

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=-30, ha="right",
             rotation_mode="anchor")

    # Turn spines off and create white grid.
    for edge, spine in ax.spines.items():
        spine.set_visible(False)

    ax.set_xticks(np.arange(data.shape[1]+1)-.5, minor=True)
    ax.set_yticks(np.arange(data.shape[0]+1)-.5, minor=True)
    ax.grid(which="minor", color="w", linestyle='-', linewidth=3)
    ax.tick_params(which="minor", bottom=False, left=False)

    return im, cbar


def annotate_heatmap(im, data=None, valfmt="{x:.2f}",
                     textcolors=["black", "white"],
                     threshold=None, **textkw):
    '''
    A function to annotate a heatmap.
    Parameters
    ----------
    im
        The AxesImage to be labeled.
    data
        Data used to annotate.  If None, the image's data is used.  Optional.
    valfmt
        The format of the annotations inside the heatmap.  This should either
        use the string format method, e.g. "$ {x:.2f}", or be a
        `matplotlib.ticker.Formatter`.  Optional.
    textcolors
        A list or array of two color specifications.  The first is used for
        values below a threshold, the second for those above.  Optional.
    threshold
        Value in data units according to which the colors from textcolors are
        applied.  If None (the default) uses the middle of the colormap as
        separation.  Optional.
    **kwargs
        All other arguments are forwarded to each call to `text` used to create
        the text labels.
    '''

    if not isinstance(data, (list, np.ndarray)):
        data = im.get_array()

    # Normalize the threshold to the images color range.
    if threshold is not None:
        threshold = im.norm(threshold)
    else:
        threshold = im.norm(data.max())/2.

    # Set default alignment to center, but allow it to be
    # overwritten by textkw.
    kw = dict(horizontalalignment="center",
              verticalalignment="center")
    kw.update(textkw)

    # Get the formatter in case a string is supplied
    if isinstance(valfmt, str):
        valfmt = matplotlib.ticker.StrMethodFormatter(valfmt)

    # Loop over the data and create a `Text` for each "pixel".
    # Change the text's color depending on the data.
    texts = []
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            kw.update(color=textcolors[int(im.norm(data[i, j]) > threshold)])
            text = im.axes.text(j, i, valfmt(data[i, j], None), **kw)
            texts.append(text)

    return texts

[Back to Table of Contents](#toc)

# Feature Selection <a class="anchor" id="feat"></a>


## Create Normalized Main Features and Evaluate Collinearity

Main Features:
* normalization = (centered at 0 and with standard deviation 1) 
* The subject characteristics will be age, gender, and diagnosis. 
* The trial parameters pertaining to the current trial will an indicator (with 1 if for this trial, choosing to gamble will always yield more money than choosing not to gamble), current expected reward (average of not gamble reward option and two gamble reward options), and gambling range (difference between higher gamble reward option and lower gamble reward option). 
* The trial parameters pertaining to the outcomes of past trials are an exponential sum of past reward prediction errors and an exponential sum of past outcomes. To elaborate, reward prediction error is actual reward outcome - expected reward outcome while the exponential sums are weighed so more recent trials have large weights closer to 1 and more previous trials have smaller weights closer to 0.

Output: 
* x and y csv files where x contains main features to predict gambling probability (0 - 1) and y contains the target gambles themselves (1's or 0's)

[Back to Table of Contents](#toc)

### define helper functions <a class="anchor" id="define-helper-functions-1"></a>

In [ ]:
# input single value
# convert ages 12 to 18
# to ages -3 to 3
def age(v):
    return v - 15

# input single value
# convert 1 (female) to 0
# convert 2 (male) to 1
def gender(v):
    return v-1

# input single value
# convert 1 (depression) to 1
# convert 2 (healthy) to 0
def diagnosis(v):
    nv = 0
    if v == 1:
        nv = 1
    return nv

def bool2int(b):
    i = 1 if b else 0
    return i


def ems(rewards, gamma):
    pastrewards = np.zeros(len(rewards))
    for i in range(len(rewards) - 1):
        pastrewards[i+1] = gamma * pastrewards[i] + rewards.iloc[i]
    return pastrewards

# input a single value
# return the utility
# as sigmoid function centered at 0 instead 0.5
def utility(v):
    u = math.exp(v) / (math.exp(v) + 1) - 0.5
    return u


[Back to Table of Contents](#toc)

### generate and save features

In [ ]:
datacsvs = ["10data_for_jess_random"]
suffices = ["_random"]

standardize = True
cn = 'Actual' 

for datacsv, suffix in zip(datacsvs, suffices):
    data=pd.read_csv(datacsv + ".csv", sep=',',header='infer')
    nrows = data.shape[0]
    data['HigherOutcome'] = data[["Outcome1Amount", "Outcome2Amount"]].max(axis=1)
    data['LowerOutcome'] = data[["Outcome1Amount", "Outcome2Amount"]].min(axis=1)
    data['Mood'] = data['Happiness']
    data['diagnosis'] = data['ptype']
    
    # update subject features
    data['age'] = data['age'].copy().map(age)
    data['gender'] = data['gender'].copy().map(gender)
    data['diagnosis'] = data['diagnosis'].copy().map(diagnosis)
    data['Indicator'] = data['CertainAmount'] < data['LowerOutcome']
    data['Indicator'] = data['Indicator'].map(bool2int)
    
    # standardize raw trial parameters
    if standardize:
        v = data.loc[:,cn].copy()
        sd = np.std(v)
        print ("for ", datacsv, " sd of ", cn, " : ", str(sd))
        data['CertainAmount'] = data['CertainAmount'].copy().map(lambda x : x / sd)
        data['HigherOutcome'] = data['HigherOutcome'].copy().map(lambda x : x / sd)
        data['LowerOutcome'] = data['LowerOutcome'].copy().map(lambda x : x / sd)
        data['Actual'] = data['Actual'].copy().map(lambda x : x / sd)
        
    # calculate utility over trial parameters
    data['CertainAmountUtility'] = data['CertainAmount'].copy().map(utility)
    data['HigherOutcomeUtility'] = data['HigherOutcome'].copy().map(utility)
    data['LowerOutcomeUtility'] = data['LowerOutcome'].copy().map(utility)
    data['ActualUtility'] = data['Actual'].copy().map(utility)
    
    # compute 2nd layer values
    data['CurrentExpectedReward'] = (data['CertainAmount'] + data['HigherOutcome'] + data['LowerOutcome']) / 3
    data['ExpectedGambleOutcome'] = (data['LowerOutcome'] + data['HigherOutcome']) / 2
    data['diff'] = data['ExpectedGambleOutcome'] - data['CertainAmount']
    data['PredictionError'] = data['Gamble'] * (data['Actual'] - data['ExpectedGambleOutcome'])
    data['CertainReward'] = (1 - data['Gamble']) * data['Actual']
    data['GamblingReward'] = data['Gamble'] * data['Actual']
    data['GamblingRange'] = data['HigherOutcome'] - data['LowerOutcome']
    
    # compute 2nd layer utilities
    data['ExpectedGambleOutcomeUtility'] = (data['LowerOutcomeUtility'] + data['HigherOutcomeUtility']) / 2
    data['diffUtility'] = data['ExpectedGambleOutcomeUtility'] - data['CertainAmountUtility']
    data['PredictionErrorUtility'] = data['Gamble'] * (data['ActualUtility'] - data['ExpectedGambleOutcomeUtility'])
    data['CertainRewardUtility'] = (1 - data['Gamble']) * data['ActualUtility']
    data['GamblingRewardUtility'] = data['Gamble'] * data['ActualUtility']
    data['GamblingRangeUtility'] = data['HigherOutcomeUtility'] - data['LowerOutcomeUtility']
    data['CurrentExpectedRewardUtility'] = (data['CertainAmountUtility'] + data['HigherOutcomeUtility'] + data['LowerOutcomeUtility']) / 3
    
    # print final column names
    colnames = data.columns
    
    # update parameters that summarize the past (mood, cumulative reward, past not gamble reward, past gamble reward)
    ## record the indices of each new subject
    ID = {}
    for i in range(nrows):
        id = int(data.iloc[i,0])
        ID[id] = ID.get(id, []) + [i]
    nsubj = len(ID)
    
    ## Create a dictionary to store all subjects
    subjDF = {}
    subjs = ID.keys()
    for subj in subjs:
        subjD = data.iloc[ID[subj],:].copy()
        subjDF[subj] = subjD
    
    # compute timeseries values
    gammas = [0.0, 0.3, 0.5, 0.7, 1.0]
    for subj in subjs:
        # fill mood with most recently reported mood, otherwise pad with 0's
        subjDF[subj].loc[:,'Mood'] = subjDF[subj].loc[:,'Mood'].copy().fillna(method='ffill').fillna(0)
        # Calculate EMS features for the following features 
        for col in ['Actual', 'CertainReward', 'GamblingReward', 'PredictionError',
                    'ActualUtility', 'CertainRewardUtility', 'GamblingRewardUtility', 'PredictionErrorUtility']:
            rewards = subjDF[subj].loc[:,col].copy()
            for gamma in gammas:
                pastrewards = ems(rewards, gamma)
                subjDF[subj].loc[:,col+'EMS'+str(gamma)] = pastrewards

    # remove the first three trials of all subjects
    def strip3(df):
        newdf = df.iloc[3:,:].copy()
        return newdf
    stripsubjDF = dict(map(lambda kv: (kv[0], strip3(kv[1])), subjDF.items()))

    # put subject data into a new dataframe
    datanew = pd.concat(stripsubjDF.values())

    # save all the columns into a csv
    if standardize:
        datanew.to_csv(datacsv + "_normed_standardized_" + cn + ".csv", index=False) # don't include the indices
    else:
        datanew.to_csv(datacsv + "_normed.csv", index=False) # don't include the indices
    
    namesl = ['subject_id', 'age', 'gender', 'diagnosis', 'mood', 
              'current expected reward', 'current gambling range', 'current indicator',
              'past rewards', 'past reward prediction error']

    # modelEV
    for gamma in gammas:
        xl = ['subject_id', 'age', 'gender', 'diagnosis', 'Mood',
              'CurrentExpectedReward', 'GamblingRange', 'Indicator', 
              'ActualEMS'+str(gamma), 'PredictionErrorEMS'+str(gamma)]
        yl = ['subject_id', 'Gamble']
        x = datanew[xl].copy()
        x = x.rename(index=str, columns=dict(zip(xl, namesl))).copy()
        y = datanew[yl].copy()
        if standardize:
            x.to_csv("x" + suffix + "_normed_standardized_" + cn + "_EV" + str(gamma) + ".csv", index=False) # don't include the indices
            y.to_csv("y" + suffix + "_normed_standardized_" + cn + "_EV" + str(gamma) + ".csv", index=False) # don't include the indices
        else:
            x.to_csv("x" + suffix + "_normed_EV" + str(gamma) + ".csv", index=False) # don't include the indices
            y.to_csv("y" + suffix + "_normed_EV" + str(gamma) + ".csv", index=False) # don't include the indices

[Back to Table of Contents](#toc)

### evaluate collinearity

In [ ]:
datacsvs = ["10data_for_jess_random"]
suffices = ["_random"]
cols = ["random"]
        
fig, axes = plt.subplots(nrows=2, ncols=3, sharex=True, sharey=True, figsize=(19,15))

for ax, col in zip(axes[0], cols):
    ax.set_title(col)

j = 0
for datacsv, suffix in zip(datacsvs, suffices):
    data=pd.read_csv(datacsv + ".csv", sep=',',header='infer')
    nrows = data.shape[0]
    data['HigherOutcome'] = data[["Outcome1Amount", "Outcome2Amount"]].max(axis=1)
    data['LowerOutcome'] = data[["Outcome1Amount", "Outcome2Amount"]].min(axis=1)
    data['Mood'] = data['Happiness']
    data['diagnosis'] = data['ptype']
    
    # update subject features
    data['age'] = data['age'].copy().map(age)
    data['gender'] = data['gender'].copy().map(gender)
    data['diagnosis'] = data['diagnosis'].copy().map(diagnosis)
    data['Indicator'] = data['CertainAmount'] < data['LowerOutcome']
    
    # compute 2nd layer values
    data['CurrentExpectedReward'] = (data['CertainAmount'] + data['HigherOutcome'] + data['LowerOutcome']) / 3
    data['ExpectedGambleOutcome'] = (data['LowerOutcome'] + data['HigherOutcome']) / 2
    data['diff'] = data['ExpectedGambleOutcome'] - data['CertainAmount']
    data['PredictionError'] = data['Gamble'] * (data['Actual'] - data['ExpectedGambleOutcome'])
    data['CertainReward'] = (1 - data['Gamble']) * data['Actual']
    data['GamblingReward'] = data['Gamble'] * data['Actual']
    data['GamblingRange'] = data['HigherOutcome'] - data['LowerOutcome']
    
    # update parameters that summarize the past (mood, cumulative reward, past not gamble reward, past gamble reward)
    ## record the indices of each new subject
    ID = {}
    for i in range(nrows):
        id = int(data.iloc[i,0])
        ID[id] = ID.get(id, []) + [i]
    nsubj = len(ID)
    
    ## Create a dictionary to store all subjects
    subjDF = {}
    subjs = ID.keys()
    for subj in subjs:
        subjD = data.iloc[ID[subj],:].copy()
        subjDF[subj] = subjD
    
    # compute timeseries values
    gammas = [0.5]
    for subj in subjs:
        # fill mood with most recently reported mood, otherwise pad with 0's
        subjDF[subj].loc[:,'Mood'] = subjDF[subj].loc[:,'Mood'].copy().fillna(method='ffill').fillna(0)
        # Calculate EMS features for the following features 
        for col in ['Actual', 'CertainReward', 'GamblingReward', 'PredictionError']:
            rewards = subjDF[subj].loc[:,col].copy()
            for gamma in gammas:
                pastrewards = ems(rewards, gamma)
                subjDF[subj].loc[:,col+'EMS'+str(gamma)] = pastrewards

    # remove the first three trials of all subjects
    def strip3(df):
        newdf = df.iloc[3:,:].copy()
        return newdf
    stripsubjDF = dict(map(lambda kv: (kv[0], strip3(kv[1])), subjDF.items()))

    # put subject data into a new dataframe
    datanew = pd.concat(stripsubjDF.values())
    
    
    ## new features
    
    namesl = ['subject_id', 'age', 'gender', 'diagnosis', 'mood', 
              'current expected reward', 'current diff', 'current gambling range', 'current indicator',
              'past rewards', 'past reward prediction error']

    xl = ['subject_id', 'age', 'gender', 'diagnosis', 'Mood',
          'CurrentExpectedReward', 'diff', 'GamblingRange', 'Indicator', 
          'ActualEMS'+str(gamma), 'PredictionErrorEMS'+str(gamma)]
    yl = ['subject_id', 'Gamble']
    x = datanew[xl].copy()
    x = x.rename(index=str, columns=dict(zip(xl, namesl))).copy()
    y = datanew[yl].copy()

    print (suffix)
    im = axes[1,j].imshow(x.iloc[:,1:].copy().corr(), vmin=-1.0, vmax=1.0)
        
    print ("new features : ")
    for i in range(1, len(namesl)):
        print (i, namesl[i])
        
    j = j + 1

fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
fig.colorbar(im, cax=cbar_ax)
plt.savefig("corr.png")
plt.show()

[Back to Table of Contents](#toc)

### visualize standardization

In [ ]:
datacsvs = ["10data_for_jess_random"]
suffices = ["_random"]
cols = ["random"]

standardize = True
cn = 'Actual'

rows = ['Before', 'After']
fig, axes = plt.subplots(nrows=2, ncols=3, sharex=True, sharey=True, figsize=(15,8))
st = fig.suptitle("Trial Values Before (Top) and After (Bottom) Standardizing with " + cn, fontsize="x-large")

for ax, col in zip(axes[0], cols):
    ax.set_title(col)

    
j = 0
for datacsv, suffix in zip(datacsvs, suffices):
    # https://stackoverflow.com/questions/31726643/how-do-i-get-multiple-subplots-in-matplotlib
    
    data=pd.read_csv(datacsv + ".csv", sep=',',header='infer')
    nrows = data.shape[0]
    data['HigherOutcome'] = data[["Outcome1Amount", "Outcome2Amount"]].max(axis=1)
    data['LowerOutcome'] = data[["Outcome1Amount", "Outcome2Amount"]].min(axis=1)
    data['Mood'] = data['Happiness']
    data['diagnosis'] = data['ptype']
    
    print (suffix, str(j))
    
    axes[0,j].hist(data['HigherOutcome'], label = 'Higher Outcome')
    axes[0,j].hist(data['LowerOutcome'], label = 'Lower Outcome')
    axes[0,j].hist(data['CertainAmount'], label = 'Certain Amount')
    axes[0,j].tick_params(axis='both', which='major', pad=10)
    
    # update subject features
    data['age'] = data['age'].copy().map(age)
    data['gender'] = data['gender'].copy().map(gender)
    data['diagnosis'] = data['diagnosis'].copy().map(diagnosis)
    
    # standardize raw trial parameters
    if standardize:
        
        v = data.loc[:,cn].copy()
        sd = np.std(v)
        print ("for ", datacsv, " sd of ", cn, " : ", str(sd))
        data['CertainAmount'] = data['CertainAmount'].copy().map(lambda x : x / sd)
        data['HigherOutcome'] = data['HigherOutcome'].copy().map(lambda x : x / sd)
        data['LowerOutcome'] = data['LowerOutcome'].copy().map(lambda x : x / sd)
        data['Actual'] = data['Actual'].copy().map(lambda x : x / sd)
    
    axes[1,j].hist(data['HigherOutcome'], label = 'Higher Outcome')
    axes[1,j].hist(data['LowerOutcome'], label = 'Lower Outcome')
    axes[1,j].hist(data['CertainAmount'], label = 'Certain Amount')
    axes[1,j].tick_params(axis='both', which='major', pad=10)
    
    j = j + 1

# https://stackoverflow.com/questions/39164828/global-legend-for-all-subplots
axes.flatten()[-2].legend(loc='upper center', bbox_to_anchor=(0.5, -0.12), ncol=3)
plt.savefig('standardize_'+cn+'.png')
plt.show()

[Back to Table of Contents](#toc)

## Calculate Quadratic Features

Output:
* x csv file with interactions, npy file with all column names

In [ ]:
x

In [ ]:
1 + nf + nf * (nf-1) / 2

In [ ]:
filenames = ['x_random_normed_standardized_Actual_EV0.5']
for filename in filenames:
    names = ['age', 'gender', 'diagnosis', 'mood',
          'current expected reward', 'current gambling range', 'current indicator',
          'past rewards', 'past reward prediction error']
    nf = len(names)
    x = np.genfromtxt(filename+ '.csv', delimiter=",")
    x2 = np.zeros(shape=(x.shape[0], int(1 + nf + nf * (nf-1) / 2)))
    
    # align subject ID column
    x2[:,range(nf+1)] = x[:,range(nf+1)]
    
    counter = nf+1
    for i in range(nf):
        for j in range(i+1, nf):
            v = x[:, i+1] * x[:, j+1]
            x2[:,counter] = v
            names.append(names[i] + ' * ' + names[j])
            counter = counter + 1
    print ("done with file : ", filename)
    np.savetxt(filename+'_interactions.csv', x2, delimiter=",")

    np.save(filename+'_interactions_names.npy', names)
with open('x_stdz_interactions.csv', 'w') as f:
    w = csv.writer(f)
    w.writerows(x2)

[Back to Table of Contents](#toc)

## General Function to Standardize Features
Start: 
* any x csv file

Data Analysis: 
* standardize.py to z-score all the features

Output: 
* x csv file with interactions, npy file with all column names, all features are standardized. This is useful for stability selection which has stability results/ rankings that are very sensitive to standard deviation of features.

In [ ]:
def standardize(filename):
    '''
    Given the name of a csv file, generate a new csv with each column standardized (mean of 0, std of 1)
    Input: filename is a string corresponding to filename.csv,
    Output: filename_stdz.csv with standardized columns, the 0th column is preserved
    '''
    temp = np.genfromtxt(filename + '.csv', delimiter=",")
    ft = np.empty(shape=[temp.shape[0]-1,temp.shape[1]])
    colnames = np.empty(shape=[temp.shape[1]])
    
    with open(filename+'.csv', 'rt') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        counter = -1
        for row in reader:
            if counter >= 0:
                ft[counter, :] = row
            else:
                colnames = row
            counter = counter + 1

    csvfile.close()
    means = []
    stds = []
    for col in range(1,temp.shape[1]):

        m = ft[:,col].mean(axis=0)
        means.append(m)
        s = ft[:,col].std(axis=0)
        stds.append(s)
        if col == 1:
            pass
        if s == 0.0 :
            ft[:,col] = (ft[:,col] - m)
        else:
            ft[:,col] = (ft[:,col] - m) / s

    with open(filename + '_stdz.csv', 'w') as writeFile:
        writer = csv.writer(writeFile)
        writer.writerow(colnames)
        writer.writerows(ft)

    writeFile.close()
    return

In [ ]:
filename = 'x_random_normed_standardized_Actual_EV0.5_interactions'
standardize(filename)

[Back to Table of Contents](#toc)

## Stability Plots and Regularization Plots

Start: 
* x csv file with interactions, npy file with all column names, all features are standardized.
Data Analysis: 
* create regularization plot with the functions reggraph(x, y, NFEATS) then reggraphsave(x,y)
* create stability plot with functions 
    * stabgraph(x, y) 
    * stabgraphsavenolegend(x, y) if you don't want the legend 
    * stabgraphsave(x, y) or stabgraphsavenolegend(x, y) or stabgraphsavefeat(x, y, l) if you want a specific list of them)
Output: 
* npy files for the stability values and regression weights at different levels of L1 regularization

[Back to Table of Contents](#toc)

### define helper functions <a class="anchor" id="define-helper-functions-2"></a>

In [ ]:

# number of bootstraps
start = 0
T = 100

LAMBDAS = list(map(lambda x : math.exp(float(x) / 2) , range(-8,20,1)))

nonzero = lambda x : not (x < tolerance and x > - tolerance)

# return a vector of feature weights
def fitmodel(xs, ys, LAMBDA):
    clf = LogisticRegression(penalty = 'l1', C = 1/LAMBDA, solver='liblinear')
    clf.fit(xs, ys.ravel())
    w = clf.coef_.ravel()
    return w

# return a list with a vector of feature weights and the intercept
def fitmodelwi(xs, ys, LAMBDA):
    clf = LogisticRegression(penalty = 'l1', C = 1/LAMBDA, solver='liblinear')
    clf.fit(xs, ys.ravel())
    w = clf.coef_.ravel()
    i = clf.intercept_.ravel()
    return [w,i]

def reggraph(x, y):
    '''
    generate weights for a regularization graph of predicting y from x
    '''
    XY = getXY(ORIGsids, x, y)
    xs = XY[0]
    ys = XY[1]
    tj = time.time()
    for LAMBDA in LAMBDAS:
        ti = time.time()
        print ("time taken : ", ti - tj)
        tj = ti
        print ("lambda : {}".format(LAMBDA))
        w = fitmodel(xs, ys, LAMBDA)
        np.save("lr_origsids_lambda"+str(LAMBDA)+"_epsilon"+str(epsilon)+ suffix + filename + str(gamma)+"_weights.npy", w)
    return

def reggraphinteractions(x, y, NFEATS):
    '''
    generate weights for a regularization graph of predicting y from x
    using only the first NFEATS features of x
    '''
    # get x, only some features
    ind_a = np.load("areas_areas_indices_" + data + ".npy") # 0 indexed
    names = np.load('x_' + ET + filename + str(gamma) + '_interactions_names.npy')
    featrank = [ind + 1 for ind in ind_a] # 1 indexed
    featranknames = [names[f-1] for f in featrank]
    ml = featrank[0: NFEATS]
    xnew = subsetx(x,ml)
    # get x, y
    XY = getXY(ORIGsids, xnew, y)
    xs = XY[0]
    ys = XY[1]

    tj = time.time()
    for LAMBDA in LAMBDAS:
        ti = time.time()
        print ("time taken : ", ti - tj)
        tj = ti
        print ("lambda : {}".format(LAMBDA))
        [w,i] = fitmodelwi(xs, ys, LAMBDA)
        np.save("lr_origsids_interactions_NFEATS" + str(NFEATS) + "_lambda"+str(LAMBDA)+"_epsilon"+str(epsilon)+ suffix + filename + str(gamma) + "_overlaps"+ str(overlaps)+  "_weights.npy", w)
        np.save("lr_origsids_interactions_NFEATS" + str(NFEATS) + "_lambda"+str(LAMBDA)+"_epsilon"+str(epsilon)+ suffix + filename + str(gamma) + "_overlaps"+ str(overlaps)+"_intercept.npy", i)
    return

def reggraphsaveinteractions(ET1, ET2, NFEATS):
    '''
    read the weights for the regularization graphs ET1 and ET2
    plot the first NFEATS features of both
    including the interactions
    excluding the intercept
    '''
    
    ind_a = np.load("areas_areas_indices_" + data + ".npy") # 0 indexed
    names = np.load('x_' + ET + filename + str(gamma) + '_interactions_names.npy')
    featrank = [ind + 1 for ind in ind_a] # 1 indexed
    featranknames = [names[f-1] for f in featrank]
    ml = featrank[0: NFEATS]
    fn = featranknames[0: NFEATS]

    weights1 = np.zeros(shape=(len(LAMBDAS), NFEATS))
    weights2 = np.zeros(shape=(len(LAMBDAS), NFEATS))
    counter = 0
    for LAMBDA in LAMBDAS:
        w1 = np.load("lr_origsids_interactions_NFEATS" + str(NFEATS) + "_lambda"+str(LAMBDA)+"_epsilon"+str(epsilon)+ "_" + ET1 + filename + str(gamma)+ "_overlaps"+ str(overlaps)+"_weights.npy")
        w2 = np.load("lr_origsids_interactions_NFEATS" + str(NFEATS) + "_lambda"+str(LAMBDA)+"_epsilon"+str(epsilon)+ "_" + ET2 + filename + str(gamma)+ "_overlaps"+ str(overlaps)+"_weights.npy")
        print (w1)
        print (w2)
        weights1[counter, :] = w1
        weights2[counter, :] = w2
        counter = counter + 1
    cm = plt.get_cmap('gist_rainbow')
    fig = plt.figure()
    ax = fig.add_subplot(111)
    NUM_COLORS = NFEATS
    originalcycle = [cm(1.*i/NUM_COLORS) for i in range(NUM_COLORS)]
    doublescycle = []
    for e in originalcycle:
        doublescycle.append(e)
        doublescycle.append(e)
    markerdoublecycle = []
    for _i in range(NFEATS):
        markerdoublecycle.append('.')
        markerdoublecycle.append('+')
    doublefn = []
    for featurename in fn:
        doublefn.append(featurename + ' (' + ET1 + ')')
        doublefn.append(featurename + ' (' + ET2 + ')')
    ax.set_prop_cycle(color = doublescycle, marker = markerdoublecycle)
    for f in range(NFEATS):
        ax.plot(LAMBDAS, weights1[:,f]) # 'o'
        ax.plot(LAMBDAS, weights2[:,f]) # '+'
    ax.axhline(y=0, color='k')
    plt.xscale('log')
    plt.legend(doublefn, loc='upper right')
    plt.title("Regularization Plot")
    plt.savefig("regplotsave.png")
    plt.show()
    return

def reggraphsaveinteractionsintercept(ET1, ET2, NFEATS):
    '''
    read the weights for the regularization graphs ET1 and ET2
    plot the first NFEATS features of both
    including the interactions
    including the intercept
    '''
    ind_a = np.load("areas_areas_indices_" + data + ".npy") # 0 indexed
    names = np.load('x_' + ET + filename + str(gamma) + '_interactions_names.npy')
    featrank = [ind + 1 for ind in ind_a] # 1 indexed
    featranknames = [names[f-1] for f in featrank]
    ml = featrank[0: NFEATS]
    fn = featranknames[0: NFEATS]

    weights1 = np.zeros(shape=(len(LAMBDAS), NFEATS+1))
    weights2 = np.zeros(shape=(len(LAMBDAS), NFEATS+1))
    counter = 0
    for LAMBDA in LAMBDAS:
        i1 = np.load("lr_origsids_interactions_NFEATS" + str(NFEATS) + "_lambda"+str(LAMBDA)+"_epsilon"+str(epsilon)+ "_" + ET1 + filename + str(gamma)+"_intercept.npy")
        i2 = np.load("lr_origsids_interactions_NFEATS" + str(NFEATS) + "_lambda"+str(LAMBDA)+"_epsilon"+str(epsilon)+ "_" + ET2 + filename + str(gamma)+"_intercept.npy")
        w1 = np.load("lr_origsids_interactions_NFEATS" + str(NFEATS) + "_lambda"+str(LAMBDA)+"_epsilon"+str(epsilon)+ "_" + ET1 + filename + str(gamma)+"_weights.npy")
        w2 = np.load("lr_origsids_interactions_NFEATS" + str(NFEATS) + "_lambda"+str(LAMBDA)+"_epsilon"+str(epsilon)+ "_" + ET2 + filename + str(gamma)+"_weights.npy")
        print (w1)
        print (w2)
        weights1[counter, :] = np.concatenate((w1, i1))
        weights2[counter, :] = np.concatenate((w2, i2))
        counter = counter + 1
    cm = plt.get_cmap('gist_rainbow')
    fig = plt.figure()
    ax = fig.add_subplot(111)
    NUM_COLORS = NFEATS+1
    originalcycle = [cm(1.*i/NUM_COLORS) for i in range(NUM_COLORS)]
    doublescycle = []
    for e in originalcycle:
        doublescycle.append(e)
        doublescycle.append(e)
    markerdoublecycle = []
    for _i in range(NFEATS+1):
        markerdoublecycle.append('.')
        markerdoublecycle.append('+')
    doublefn = []
    for featurename in fn:
        doublefn.append(featurename + ' (' + ET1 + ')')
        doublefn.append(featurename + ' (' + ET2 + ')')
    doublefn.append('intercept (' + ET1 + ')')
    doublefn.append('intercept (' + ET2 + ')')
    ax.set_prop_cycle(color = doublescycle, marker = markerdoublecycle)
    for f in range(NFEATS+1):
        ax.plot(LAMBDAS, weights1[:,f]) # 'o'
        ax.plot(LAMBDAS, weights2[:,f]) # '+'
    ax.axhline(y=0, color='k')
    plt.xscale('log')
    plt.legend(doublefn, loc='upper right')
    plt.title("Regularization Plot")
    plt.savefig("regplotsave.png")
    plt.show()
    return

def reggraphsave(x, y):
    '''
    read the weights for and plot the regularization graph of predicting y from x
    '''
    XY = getXY(ORIGsids, x, y)
    xs = XY[0]
    ys = XY[1]
    weights = np.zeros(shape=(len(LAMBDAS), nf))
    counter = 0
    for LAMBDA in LAMBDAS:
        w = np.load("lr_origsids_lambda"+str(LAMBDA)+"_epsilon"+str(epsilon)+ suffix + filename + str(gamma)+"_weights.npy")
        weights[counter, :] = w
        counter = counter + 1
    for f in range(len(names)):
        plt.plot(LAMBDAS, weights[:,f])

    plt.xscale('log')
    plt.legend(names, loc='upper right')
    plt.title("Regularization Plot")
    plt.savefig("regplotsave.png")
    plt.show()
    return

def reggraphsavefeat(x, y, l):
    '''
    read the weights for and plot the regularization graph of predicting y from x
    only for features in list l
    '''
    XY = getXY(ORIGsids, x, y)
    xs = XY[0]
    ys = XY[1]
    weights = np.zeros(shape=(len(LAMBDAS), nf))
    counter = 0
    for LAMBDA in LAMBDAS:
        w = np.load("lr_origsids_lambda"+str(LAMBDA)+"_epsilon"+str(epsilon)+ suffix + filename + str(gamma)+"_weights.npy")
        weights[counter, :] = w
        counter = counter + 1
    cm = plt.get_cmap('gist_rainbow')
    fig = plt.figure()
    ax = fig.add_subplot(111)
    NUM_COLORS = len(l)
    ax.set_color_cycle([cm(1.*i/NUM_COLORS) for i in range(NUM_COLORS)])
    for f in l:
        ax.plot(LAMBDAS, weights[:,f])

    plt.xscale('log')
    plt.legend([names[l[i]] for i in range(len(l))], loc='lower right')
    plt.title("Regularization Plot")
    plt.savefig("regplotsave.png")
    plt.show()
    return

def stabgraph(x, y):
    '''
    generate the subsamples of a stability graph of predicting y from x
    '''
    index = 0
    tj = time.time()
    for LAMBDA in LAMBDAS:
        counts = np.zeros(8)
        for t in range(start, start + T):
            ti = time.time()
            print ("time taken : ", ti - tj)
            tj = ti
            index = index + 1
            print ("lambda : {} - Subsample : {}".format(LAMBDA, t))
            ns = int(len(ORIGsids) / 2)
            np.random.seed(t) ; sids = np.random.choice(ORIGsids, ns, replace=False)
            XY = getXY(sids, x, y)
            xs = XY[0]
            ys = XY[1]
            w = fitmodel(xs, ys, LAMBDA)
            np.save("lr2_sids_subsample"+str(t)+"_lambda"+str(LAMBDA)+"_epsilon"+str(epsilon)+ suffix + filename + str(gamma)+"_weights.npy", w)
    return


def stabgraphsave(x, y):
    '''
    use the subsamples of a stability graph of predicting y from x
    to save and plot the ratios with a legend
    '''
    ratios = np.zeros(shape=(len(LAMBDAS), nf))
    counter = 0
    for LAMBDA in LAMBDAS:
        all_signs = np.zeros(shape=(T, nf))
        counts = np.zeros(nf)
        for t in range(start, start+T):
            w = np.load("lr2_sids_subsample"+str(t)+"_lambda"+str(LAMBDA)+"_epsilon"+str(epsilon)+ suffix + filename + str(gamma)+"_weights.npy")
            all_signs[t - start, :] = list(map(np.sign, w))
        pos_sum = np.sum(all_signs == 1, 0)
        neg_sum = np.sum(all_signs == -1, 0)
        counts = np.maximum(pos_sum, neg_sum)
        ratios[counter, :] = list(map(lambda x : float(x) / float(T), counts))
        counter = counter + 1
    plt.figure()
    for f in range(len(names)):
        plt.plot(LAMBDAS, ratios[:,f])
    np.save("lr_sids_ratios_for_alllambdas_of_" + ET + suffix + filename + str(gamma) + ".npy", ratios)
    np.save("lr_sids_lambdas_for_alllambdas_of_" + ET + suffix + filename + str(gamma) + ".npy", LAMBDAS)
    plt.legend(names, loc='lower left')
    plt.title("Stability Plot")
    plt.xscale('log')
    plt.savefig("stabgplotsave.png")
    plt.show()
    return

def stabgraphsavenolegend(x, y):
    '''
    use the subsamples of a stability graph of predicting y from x
    to save and plot the ratios with a legend
    '''
    ratios = np.zeros(shape=(len(LAMBDAS), nf))
    counter = 0
    for LAMBDA in LAMBDAS:
        all_signs = np.zeros(shape=(T, nf))
        counts = np.zeros(nf)
        for t in range(start, start+T):
            w = np.load("lr2_sids_subsample"+str(t)+"_lambda"+str(LAMBDA)+"_epsilon"+str(epsilon)+ suffix + filename + str(gamma)+"_weights.npy")
            all_signs[t - start, :] = list(map(np.sign, w))
        pos_sum = np.sum(all_signs == 1, 0)
        neg_sum = np.sum(all_signs == -1, 0)
        counts = np.maximum(pos_sum, neg_sum)
        ratios[counter, :] = list(map(lambda x : float(x) / float(T), counts))
        counter = counter + 1
    plt.figure()
    for f in range(len(names)):
        plt.plot(LAMBDAS, ratios[:,f])
    np.save("lr_sids_ratios_for_alllambdas_of_" + ET + suffix + filename + str(gamma)+ ".npy", ratios)
    np.save("lr_sids_lambdas_for_alllambdas_of_" + ET + suffix + filename + str(gamma)+ ".npy", LAMBDAS)
    plt.title("Stability Plot")
    plt.xscale('log')
    plt.savefig("stabgplotsave.png")
    plt.show()
    return

def stabgraphsavefeat(x, y, l):
    '''
    use the subsamples of a stability graph of predicting y from x
    to save and plot ratios with only for x's with indices in the list l (0-indexed)
    '''
    ratios = np.zeros(shape=(len(LAMBDAS), nf))
    counter = 0
    for LAMBDA in LAMBDAS:
        all_signs = np.zeros(shape=(T, nf))
        counts = np.zeros(nf)
        for t in range(start, start+T):
            w = np.load("lr2_sids_subsample"+str(t)+"_lambda"+str(LAMBDA)+"_epsilon"+str(epsilon)+ suffix + filename + str(gamma)+"_weights.npy")
            all_signs[t - start, :] = list(map(np.sign, w))
        pos_sum = np.sum(all_signs == 1, 0)
        neg_sum = np.sum(all_signs == -1, 0)
        counts = np.maximum(pos_sum, neg_sum)
        ratios[counter, :] = list(map(lambda x : float(x) / float(T), counts))
        counter = counter + 1
    cm = plt.get_cmap('gist_rainbow')
    fig = plt.figure()
    ax = fig.add_subplot(111)
    NUM_COLORS = len(l)
    ax.set_color_cycle([cm(1.*i/NUM_COLORS) for i in range(NUM_COLORS)])
    for f in l:
        ax.plot(LAMBDAS, ratios[:,f])
    namesl = [names[l[i]] for i in range(len(l))]
    np.save("lr_sids_ratios_for_alllambdas_of_" + ET + suffix + filename + str(gamma)+ ".npy", ratios)
    np.save("lr_sids_lambdas_for_alllambdas_of_" + ET + suffix + filename + str(gamma)+ ".npy", LAMBDAS)
    plt.legend(namesl, loc='lower left')
    plt.title("Stability Plot")
    plt.xscale('log')
    plt.savefig("stabgplotsave.png")
    plt.show()

    return



[Back to Table of Contents](#toc)

### create plots 

In [ ]:
overlaps = False

epsilon=0.0
tolerance = 0.0005
it = "no interactions"

# where to get features from
data = "random"
NFEATS = 7

# where to build model
ET = "random"
suffix = '_' + ET
filename = '_normed_standardized_Actual_EV'
gamma = 0.5


if it == "no interactions":
    x = np.genfromtxt("x" + suffix + filename + str(gamma) + ".csv", delimiter=",")
    i = getcolumnnames("x" + suffix + filename + str(gamma) + ".csv")
    names = i[1:]
    print (names)
    nf = len(names)
elif it == "with interactions":
    if overlaps:
        x = np.genfromtxt('x' + suffix + filename + str(gamma) + '_interactions_stdz_overlap.csv', delimiter=",")
    else:
        x = np.genfromtxt('x' + suffix + filename + str(gamma) + '_interactions_stdz.csv', delimiter=",")
    names = np.load('x' + suffix + filename + str(gamma) + '_interactions_names.npy')
    nf = len(names)
else:
    print ("it parameter was not recognized")

if overlaps:
    y = np.genfromtxt("y" + suffix + filename + str(gamma) + "_overlap.csv", delimiter=",")
else:
    y = np.genfromtxt("y" + suffix + filename + str(gamma) + ".csv", delimiter=",")
# select gps
if ET == "1block":
    gps = 'gps.csv'
elif ET == "3block":
    gps = 'gps_3block.csv'
elif ET == "random":
    gps = 'gps_random.csv'
else:
    print ("error")
sf = np.genfromtxt(gps, delimiter=",", usecols=np.arange(0,2)) # subject file
ORIGsids = np.array(sf[1:,1]) # subject ids
n = len(ORIGsids)

stabgraph(x, y)
stabgraphsavenolegend(x, y)
reggraph(x,y)

[Back to Table of Contents](#toc)

## Average Stability Calculations

Start: 
* npy files for the stability values at different levels of L1 regularization

Output: 
* csv file with the names and average stabilities of all features (averaged across lambda values), npy file with ranking of indices of the top most stable (on average) features


### save stability values

In [ ]:
ET = "random"
it = "no interactions"
filename = '_normed_standardized_Actual_EV'
# filename = '_normed_EV'
gamma = 0.5

suffix = "_"+ET

ratios = np.load("lr_sids_ratios_for_alllambdas_of_" + ET + suffix + filename + str(gamma) + ".npy")
print (ratios.shape)
LAMBDAS = np.load("lr_sids_lambdas_for_alllambdas_of_" + ET + suffix + filename + str(gamma) + ".npy")
print (LAMBDAS.shape)


if it == "no interactions":
    i = getcolumnnames("x_" + ET + filename + str(gamma) + ".csv")
    names = i[1:]
elif it == "with interactions":
    print ("file : " + 'x' + suffix + filename + str(gamma) + '_interactions_names.npy')
    names = np.load('x' + suffix + filename + str(gamma) + '_interactions_names.npy')
    print (names)
else:
    print ("it parameter was not recognized")

# compute area
# x is a list of increments along the x axis
# y is a list of highs along each increment
# left trapezoidal rules
def ca(x,y):
    area = 0.0
    for i in range(len(x) - 1):
        dx = x[i+1] - x[i]
        dy = y[i+1] - y[i]
        area = area + dx * y[i+1] + dx * dy / 2
    maxarea = max(y) * (x[-1] - x[0]) # use this if you want to normalize
    area = area
    return area

areas = np.zeros(shape=len(names))
maxes = np.zeros(shape=len(names))
with open('areas_' + ET + suffix + filename + str(gamma) + '.csv', 'a') as file:
    filewriter = csv.writer(file, delimiter=',')
    filewriter.writerow(['index', 'name','area','max'])
    counter = 0
    for f in range(len(names)):
        fname = names[f]
        fratios = ratios[:,f]
        fa = ca(LAMBDAS, fratios)
        fm = np.max(fratios)
        filewriter.writerow([counter, fname, fa, fm])
        areas[counter] = fa
        maxes[counter] = fm
        counter = counter + 1
    np.save("areas_areas_" + ET + suffix + filename + str(gamma) + ".npy", areas)
    np.save("areas_maxes_" + ET + suffix + filename + str(gamma) + ".npy", maxes)
    file.close()

[Back to Table of Contents](#toc)

### save indicies of stability values

In [ ]:
ET = "random"
filename = '_normed_standardized_Actual_EV'
gamma = 0.5
suffix = "_"+ET

areas = np.load("areas_areas_" + ET + suffix + filename + str(gamma)+ ".npy")
maxes = np.load("areas_maxes_" + ET + suffix + filename + str(gamma)+ ".npy")
# sort indices based on largest to smallest area
ind_a = np.flip(np.argsort(areas))
# sort indices based on largest to smallest max stability
ind_m = np.flip(np.argsort(maxes))

np.save("areas_areas_indices_" + ET + suffix + filename + str(gamma)+ ".npy", ind_a)
np.save("areas_maxes_indices_" + ET + suffix + filename + str(gamma)+ ".npy", ind_m)

[Back to Table of Contents](#toc)

## Picking Number of Most Stable Features

Start: 
* ranking of indices of most stable features

Output: 
* Graph of LOOCV accuracy of models trained on 0 features, 1 top most stable feature, top 2 most stable features, top 3 most stable features, ..., all the way to 45 features (9 main features + 36 interactions). 
* We can pick a sufficient number of features by seeing which # of features achieves the global maximum LOOCV accuracy (by leaving out one subject instead of one trial)

In [ ]:
it = "no interactions"

ET = "random"
suffix = '_' + ET
filename = '_normed_standardized_Actual_EV'
gamma = 0.5

# x and y myst both have a subject_id column
y = np.genfromtxt("y" + suffix + filename + str(gamma) + ".csv", delimiter=",")
sf = np.genfromtxt('gps' + suffix + '.csv', delimiter=",", usecols=np.arange(0,2)) # subject file
ORIGsids = np.array(sf[1:,1]) # subject ids
n = len(ORIGsids)

# number of bootstraps
T = 100
graphtype = "m" 


# x and y both have a subject_id column
# return accuracy of a logistic regression model
# through LOOCV
def loocv_acc(sids, x, y):
    # obsvs = np.zeros(shape=(n,nTRIALS))
    # preds = np.zeros(shape=(n,nTRIALS))
    obsvs = []
    preds = []
    probs = []
    print (preds)
    if len(x) == 0:
        for f in range(n):
            test_y = subjY(sids[f], y)
            train_y = notsubjY(sids[f], y)
            ym = np.mean(train_y )
            yb = ym > 0.5
            test_p = [yb for _i in range(len(test_y))]
            test_prob = [ym for _i in range(len(test_y))]
            preds.extend(test_p)
            obsvs.extend(test_y)
            probs.extend(test_prob)
    else:
        for f in range(n):
            if sids[f] > 10:
                [test_x, test_y] = subjXY(sids[f], x, y)
                [train_x, train_y] = notsubjXY(sids[f], x, y)
                if x.shape[1] == 2:
                    test_x = test_x.reshape(len(test_x),1)
                    train_x = train_x.reshape(len(train_x),1)
                mf = LogisticRegression(solver='liblinear') # penalty = 'l1', C = 1/LAMBDA,
                mf.fit(train_x, train_y)
                test_p = mf.predict(test_x)
                temp = mf.predict_proba(test_x)
                test_prob = np.reshape(np.array(temp[:,1]), (-1))
                preds.extend(test_p)
                obsvs.extend(test_y)
                probs.extend(test_prob)
    predsbool = np.reshape(np.array(preds).astype(bool), (-1))
    obsvsbool = np.reshape(np.array(obsvs).astype(bool), (-1))
    probs = np.reshape(np.array(probs), (-1))
    bacc = balancedacc(obsvsbool, probs, 0.5)
    print ("balanced accuracy is : ", bacc)
    fpr, tpr, thresholds = sklearn.metrics.roc_curve(obsvsbool, probs)
    auc = sklearn.metrics.auc(fpr, tpr)
    return [bacc, auc]

if it == "no interactions":
    x = np.genfromtxt("x" + suffix + filename + str(gamma) + ".csv", delimiter=",")
    i = getcolumnnames("x" + suffix + filename + str(gamma) + ".csv")
    names = i[1:]
    # indices in x of the most stable features
    ind_a = np.load("areas_areas_indices_" + ET + suffix + filename + str(gamma) + ".npy") # 0 indexed
    
    featrank = [ind + 1 for ind in ind_a] # 1 indexed
    featranknames = [names[f-1] for f in featrank]
elif it == "with interactions":
    x = np.genfromtxt('x' + suffix + filename + str(gamma) + '_interactions.csv', delimiter=",")
    names = np.load('x' + suffix + filename + str(gamma) + '_interactions_names.npy')
    ind_a = np.load("areas_areas_indices_"+ET+ suffix + filename + str(gamma)+".npy")
    featrank = [ind_a[i]+1 for i in range(len(ind_a))]
    featranknames = [names[f-1] for f in featrank]
else:
    print ("it parameter was not recognized")


baccs = np.zeros(shape=(T, len(featrank) + 1))
aucs = np.zeros(shape=(T, len(featrank) + 1))
for t in range(T):
    if t == 0:
        sids = ORIGsids
    else:
        np.random.seed(t) ; sids = np.random.choice(ORIGsids, n, replace=True)
    [mbacc,auc] = loocv_acc(sids, [], y)
    baccs[t,0] = mbacc
    aucs[t,0] = auc
    for m in range(len(featrank)):
        ml = featrank[0: m+1]
        xnew = subsetx(x,ml)
        [mbacc, mauc] = loocv_acc(sids, xnew, y)
        baccs[t, m+1] = mbacc
        aucs[t, m+1] = mauc
np.save("bmaccs_" + suffix + filename + str(gamma) + ".npy", bmaccs)
np.save("aucs_" + suffix + filename + str(gamma) + ".npy", aucs)
accs = np.load("accs_" + suffix + filename + str(gamma) + ".npy")
print (accs)

baccs = np.load("baccs_" + suffix + filename + str(gamma) + ".npy")
aucs = np.load("aucs_" + suffix + filename + str(gamma) + ".npy")

print ("experiment type is : ", ET)

plt.figure()
plt.xlabel("Number of Features")
plt.ylabel("Max Balanced Accuracy in LOOCV")
x = ["Baseline"]
for name in featranknames:
    x.append("+ " + name)

if graphtype == "all":
    ### # all bootstraps
    for t in range(T):
        plt.plot(x, baccs[t,:])
    plt.title("Each line is for a bootstrap resampling T = " + str(T) + " bootstrap resamplings")
elif graphtype == "m":
    ### # just the means
    plt.plot(x, baccs.mean(axis=0))
    for i, txt in enumerate(baccs.mean(axis=0)):
        plt.annotate(round(txt,2), (x[i], baccs.mean(axis=0)[i]))
    plt.title("Mean Balanced Accuracy for T = " + str(T) + " bootstrap resamplings for " + ET)
elif graphtype == "ms":
    ### # just the means and stds
    plt.errorbar(x, baccs.mean(axis=0), yerr = baccs.std(axis=0))
    for i, txt in enumerate(baccs.mean(axis=0)):
        plt.annotate(round(txt,2), (x[i], baccs.mean(axis=0)[i]))
    plt.title("Means and STDs for T = " + str(T) + " bootstrap resamplings for " + ET)
elif graphtype == "one":
    plt.plot(x, baccs[0,:])
    for i, txt in enumerate(baccs[0,:]):
        plt.annotate(round(txt,2), (x[i], baccs[0,:][i]))
    plt.title("Only for Original Sample")
else:
    print ("graphtype parameter not recognized")
plt.show()

plt.figure()
plt.xlabel("Number of Features")
plt.ylabel("AUC in LOOCV")
plt.plot(x,aucs.mean(axis=0))
for i, txt in enumerate(aucs.mean(axis=0)):
    plt.annotate(round(txt,2), (x[i], aucs.mean(axis=0)[i]))
plt.title("Means and STDs for T = " + str(T) + " bootstrap resamplings for " + ET)

plt.show()

[Back to Table of Contents](#toc)

# Logistic Regression Modeling and Interpretation

## Estimate LOOCV accuracy (leaving out one subject's 90 trials)

Output: 
* csv file of results

In [ ]:
def runmodel(gps, model, NFEATS, x, y, SEED, LAMBDA):
    '''
    Generate and evaluate the predictions of a model with the following inputs:

    gps is the name of the per subject data gps file (see above)
    model is a string denoting the type of model
    x is an array of features, with the 0th column as subject ids
    y is an array of targets, with the 0th column as subject ids
    '''
    
    #ind_a = np.load("areas_areas_indices_" + data + ".npy") # 0 indexed
    ind_a = np.load("areas_areas_indices_" + ET + suffix + filename + str(gamma) + ".npy") # 0 indexed
    featrank = [ind + 1 for ind in ind_a] # 1 indexed
    featranknames = [names[f-1] for f in featrank]
    ml = featrank[0: NFEATS]
    xnew = subsetx(x,ml)

    sf = np.genfromtxt(gps, delimiter=",",usecols=np.arange(0,2)) # subject file
    ORIGsids = np.array(sf[1:,1]) # subject ids
    ORIGsids = list(filter(lambda x: x > 10, ORIGsids))
    n = len(ORIGsids)
    if SEED == 0:
        sids = ORIGsids
    else:
        np.random.seed(SEED) ; sids = np.random.choice(ORIGsids, n)
    obsvs = []
    preds = []
    probs = []
    subjs = []
    for fold in range(n):
        if model == model == "topfeatures":
            [test_x, test_y] = subjXY(sids[fold], xnew, y)
            [train_x, train_y] = notsubjXY(sids[fold], xnew, y)
            mf = LogisticRegression(solver='liblinear', penalty = 'l1', C = 1/LAMBDA)
            mf.fit(train_x, train_y)
            test_pred = mf.predict(test_x)
            temp = mf.predict_proba(test_x)
            test_prob = np.reshape(np.array(temp[:,1]), (-1))
        elif model == "allfeatures"  or model == "top2features" or model == "top3features" or model == "top5features" or model == "top10features":
            [test_x, test_y] = subjXY(sids[fold], x, y)
            [train_x, train_y] = notsubjXY(sids[fold], x, y)
            mf = LogisticRegression(solver='liblinear') # penalty = 'l1', C = 1/LAMBDA,
            mf.fit(train_x, train_y)
            test_pred = mf.predict(test_x)
            temp = mf.predict_proba(test_x)
            test_prob = np.reshape(np.array(temp[:,1]), (-1))
        elif model == "baseline":
            test_y = subjY(sids[fold], y)
            train_y = notsubjY(sids[fold], y)
            ym = np.mean(train_y)
            yb = ym > 0.5
            test_pred = [yb for _i in range(len(test_y))]
            if yb == 1:
                print ("predict 1 always with probability ", ym)
                test_prob = [ym for _i in range(len(test_y))]
            else:
                print ("predict 0 always with probability ", 1- ym)
                test_prob = [1- ym for _i in range(len(test_y))]
        else:
            print ("model parameter not recognized")
        preds.append(test_pred)
        obsvs.append(test_y)
        probs.append(test_prob)
        subjs.append([sids[fold] for i in range(len(test_pred))])
    subjs = np.array(subjs).reshape(-1)
    probs = np.array(probs).reshape(-1)
    predsbool = np.reshape(np.array(preds).astype(bool), (-1))
    obsvsbool = np.reshape(np.array(obsvs).astype(bool), (-1))
    bacc3 = balancedacc(obsvsbool, probs, 0.3)
    bacc5 = balancedacc(obsvsbool, probs, 0.5)
    bacc7 = balancedacc(obsvsbool, probs, 0.7)
    mbacc = maxbacc(obsvsbool, probs)
    fpr, tpr, thresholds = sklearn.metrics.roc_curve(obsvsbool, probs)
    print ("")
    print ("for model ", model)
    auc = sklearn.metrics.auc(fpr, tpr)
    sauc = subjauc(subjs, obsvsbool, probs)
    with open('predictions_'+data+'.csv', 'w') as writeFile:
        writer = csv.writer(writeFile)
        writer.writerow(["subjid", "truth", "prob"])
        writer.writerows(zip(subjs,obsvsbool, probs))
    writeFile.close()
    ll = sklearn.metrics.log_loss(np.squeeze(obsvsbool), np.squeeze(probs)) / float(len(probs))
    print ("balanced accuracy is : ", round(bacc5,4))
    print ("auc is : ", round(auc,4))
    print ("log loss is : ", '%.4E' % Decimal(ll))
    return [bacc3, bacc5, bacc7, mbacc, fpr, tpr, auc, sauc, ll]

## Decide where features are coming from
ET = "random"
specificNFEATS = 45
ETsuffix = "_" + ET
## Decide what data to test in
data = "random"
suffix = "_"+data

# select gps
if data == "1block":
    gps = 'gps.csv'
elif data == "3block":
    gps = 'gps_3block.csv'
elif data == "random":
    gps = 'gps_random.csv'
else:
    print ("error")

filename = '_normed_standardized_Actual_EV' 
gamma = 0.5
LAMBDA = 1.0
LAMBDAS = [0.1, 1.0, 10.0]
nSEEDS = 3

ind_a = np.load("areas_areas_indices_" + ET + ETsuffix + filename + str(gamma)+ ".npy")
names = np.load('x_' + ET + filename + str(gamma) + '_interactions_names.npy')


models = ["topfeatures"] 
NFEATSs = [len(ind_a)] 

x = np.genfromtxt("x" + suffix + filename + str(gamma) + "_interactions.csv", delimiter=",")
y = np.genfromtxt("y" + suffix + filename + str(gamma) + ".csv", delimiter=",")

with open('lr_results.csv', 'w') as writeFile:
    writer = csv.writer(writeFile)
    results = []
    for SEED in range(nSEEDS):
        for LAMBDA in LAMBDAS:
            for model, NFEATS in zip(models, NFEATSs):
                [bacc3, bacc5, bacc7, mbacc, fpr, tpr, auc, sauc, ll] = runmodel(gps, model, NFEATS, x, y, SEED, LAMBDA)
                writer.writerow([bacc5, str(LAMBDA), str(SEED)])
                results.append([bacc5, str(LAMBDA), str(SEED)])


[Back to Table of Contents](#toc)

## Estimate Transfer Accuracy

Output
* csv file of results

In [ ]:
# where to get your features from
ET = "random"
specificNFEATS = 7
it = "no interactions"
filename = '_normed_standardized_Actual_EV'
gamma = 0.5

suffix = "_"+ET

print ("areas_areas_indices_" + ET + suffix + filename + str(gamma)+ ".npy")
ind_a = np.load("areas_areas_indices_" + ET + suffix + filename + str(gamma)+ ".npy") # 0 indexed
featrank = [ind + 1 for ind in ind_a] # 1 indexed
colors = ['aqua', 'darkorange', 'cornflowerblue']


# train model on only subjects that overlap between train and test set
# test model on only subjects also in train set
def runmodeloverlap2(model, traintype, testtype, NFEATS):

    # find overlap
    trainsubj = np.genfromtxt('y'+str(trainsuffix)+filename1+'.csv', delimiter=',')[1:,0]
    testsubj = np.genfromtxt('y'+str(testsuffix)+filename1+'.csv', delimiter=',')[1:,0]

    overlapsubj = list(set(trainsubj) & set(testsubj))
    print (str(len(overlapsubj)) + " overlap subjects between " + traintype + " and " + testtype)
    print (overlapsubj)

    # create gps file
    rows = zip(range(len(overlapsubj)), overlapsubj)
    with open('gps_overlap_'+ traintype + '_' + testtype + '.csv', 'w') as writeFile:
        writer = csv.writer(writeFile)
        writer.writerow(['index', 'subject_id'])
        writer.writerows(rows)
    writeFile.close()

    train_y = subsetfilesubj('y'+str(trainsuffix)+filename1, overlapsubj)
    test_y = subsetfilesubj('y'+str(testsuffix)+filename1, overlapsubj)
    subjs = subsetfilesubjSUBJS('y'+str(testsuffix)+filename1, overlapsubj)

    # define variables
    np.save("test_obsv.npy", test_y)
    if model == "topfeatures":
        train_x = strip(subsetxfile('x'+str(trainsuffix)+filename2+'_overlap_frm'+ testtype +'.csv', featrank[0:NFEATS]))
        _temp = subsetfilesubj('x'+str(testsuffix)+filename2, overlapsubj)
        test_x = strip(subsetxfile('x'+str(testsuffix) +filename2+ '_overlap_frm'+ traintype +'.csv', featrank[0:NFEATS]))
    elif model == "baseline":
        pass
    else:
        print ("model parameter not recognized")

    # create model and start predictions
    if model == "topfeatures":
        mf = LogisticRegression(solver='liblinear')
        mf.fit(train_x, train_y)
        test_pred = mf.predict(test_x)
        temp = mf.predict_proba(test_x)
        test_prob = np.reshape(np.array(temp[:,1]), (-1))
    elif model == "baseline":
        ym = np.mean(train_y)
        yb = ym > 0.5
        test_pred = [yb for _i in range(len(test_y))]
        if yb == 1:
            print ("predict 1 always with probability ", ym)
            test_prob = [ym for _i in range(len(test_y))]
        else:
            print ("predict 0 always with probability ", 1- ym)
            test_prob = [1- ym for _i in range(len(test_y))]
    else:
        print ("model parameter not recognized")

    test_prob = np.reshape(np.array(test_prob), (-1))
    predsbool = np.reshape(np.array(test_pred).astype(bool), (-1))
    obsvsbool = np.reshape(np.array(test_y).astype(bool), (-1))

    bacc3 = balancedacc(obsvsbool, test_prob, 0.3)
    bacc5 = balancedacc(obsvsbool, test_prob, 0.5)
    bacc7 = balancedacc(obsvsbool, test_prob, 0.7)
    mbacc = maxbacc(obsvsbool, test_prob)
    fpr, tpr, thresholds = sklearn.metrics.roc_curve(obsvsbool, test_prob)
    print ("")
    print ("for model ", model)
    sauc = subjauc(subjs, obsvsbool, test_prob)
    auc = sklearn.metrics.auc(fpr, tpr)
    ll = sklearn.metrics.log_loss(np.squeeze(obsvsbool), np.squeeze(test_prob)) / float(len(test_prob))
    print ("balanced accuracy is : ", round(bacc5,4))
    print ("auc is : ", round(auc,4))
    print ("log loss is : ", '%.4E' % Decimal(ll))
    return [bacc3, bacc5, bacc7, mbacc, fpr, tpr, auc, sauc, ll]


# train model on full train set
# test model on only subjects also in train set
def runmodeloverlap(model, traintype, testtype, NFEATS):
    # define variables
    train_y = stripfile('y'+str(trainsuffix)+filename1+'.csv')

    # find overlap
    trainsubj = np.genfromtxt('y'+str(trainsuffix)+filename1+'.csv', delimiter=',')[1:,0]
    testsubj = np.genfromtxt('y'+str(testsuffix)+filename1+'.csv', delimiter=',')[1:,0]

    overlapsubj = list(set(trainsubj) & set(testsubj))
    print (str(len(overlapsubj)) + " overlap subjects between " + traintype + " and " + testtype)
    print (overlapsubj)

    # create gps file
    rows = zip(range(len(overlapsubj)), overlapsubj)
    with open('gps_overlap_'+ traintype + '_' + testtype + '.csv', 'w') as writeFile:
        writer = csv.writer(writeFile)
        writer.writerow(['index', 'subject_id'])
        writer.writerows(rows)
    writeFile.close()

    test_y = subsetfilesubj('y'+str(testsuffix)+filename1, overlapsubj)
    subjs = subsetfilesubjSUBJS('y'+str(testsuffix)+filename1, overlapsubj)

    # define variables
    np.save("test_obsv.npy", test_y)
    if model == "topfeatures":
        train_x = strip(subsetxfile('x'+str(trainsuffix)+filename2+'.csv', featrank[0:NFEATS]))
        _temp = subsetfilesubj('x'+str(testsuffix)+filename2, overlapsubj)
        test_x = strip(subsetxfile('x'+str(testsuffix) +filename2+ '_overlap_frm'+ traintype +'.csv', featrank[0:NFEATS]))
    elif model == "baseline":
        pass
    else:
        print ("model parameter not recognized")

    # create model and start predictions
    if model == "topfeatures":
        mf = LogisticRegression(solver='liblinear')
        mf.fit(train_x, train_y)
        test_pred = mf.predict(test_x)
        temp = mf.predict_proba(test_x)
        test_prob = np.reshape(np.array(temp[:,1]), (-1))
    elif model == "baseline":
        ym = np.mean(train_y)
        yb = ym > 0.5
        test_pred = [yb for _i in range(len(test_y))]
        if yb == 1:
            print ("predict 1 always with probability ", ym)
            test_prob = [ym for _i in range(len(test_y))]
        else:
            print ("predict 0 always with probability ", 1- ym)
            test_prob = [1- ym for _i in range(len(test_y))]
    else:
        print ("model parameter not recognized")

    test_prob = np.reshape(np.array(test_prob), (-1))
    predsbool = np.reshape(np.array(test_pred).astype(bool), (-1))
    obsvsbool = np.reshape(np.array(test_y).astype(bool), (-1))

    bacc3 = balancedacc(obsvsbool, test_prob, 0.3)
    bacc5 = balancedacc(obsvsbool, test_prob, 0.5)
    bacc7 = balancedacc(obsvsbool, test_prob, 0.7)
    mbacc = maxbacc(obsvsbool, test_prob)
    fpr, tpr, thresholds = sklearn.metrics.roc_curve(obsvsbool, test_prob)
    print ("")
    print ("for model ", model)
    sauc = subjauc(subjs, obsvsbool, test_prob)
    auc = sklearn.metrics.auc(fpr, tpr)
    ll = sklearn.metrics.log_loss(np.squeeze(obsvsbool), np.squeeze(test_prob)) / float(len(test_prob))
    print ("balanced accuracy is : ", round(bacc5,4))
    print ("auc is : ", round(auc,4))
    print ("log loss is : ", '%.4E' % Decimal(ll))
    return [bacc3, bacc5, bacc7, mbacc, fpr, tpr, auc, sauc, ll]

def runmodel(model, traintype, testtype, NFEATS):
    # define variables
    train_y = stripfile('y'+str(trainsuffix)+filename1+'.csv')
    test_y = stripfile('y'+str(testsuffix)+filename1+'.csv')
    subjs = np.genfromtxt('y'+str(testsuffix)+filename1+'.csv', delimiter=",")[1:,0]
    np.save("test_obsv.npy", test_y)
    if model == "topfeatures":
        train_x = strip(subsetxfile('x'+str(trainsuffix) + filename2 +'.csv', featrank[0:NFEATS]))
        test_x = strip(subsetxfile('x'+str(testsuffix) + filename2 +'.csv', featrank[0:NFEATS]))
    elif model == "baseline":
        pass
    else:
        print ("model parameter not recognized")

    # create model and start predictions
    if model == "topfeatures":
        mf = LogisticRegression(solver='liblinear', penalty = 'l1', C = 1/LAMBDA)
        mf.fit(train_x, train_y)
        test_pred = mf.predict(test_x)
        temp = mf.predict_proba(test_x)
        test_prob = np.reshape(np.array(temp[:,1]), (-1))
    elif model == "baseline":
        ym = np.mean(train_y)
        yb = ym > 0.5
        test_pred = [yb for _i in range(len(test_y))]
        if yb == 1:
            print ("predict 1 always with probability ", ym)
            test_prob = [ym for _i in range(len(test_y))]
        else:
            print ("predict 0 always with probability ", 1- ym)
            test_prob = [1- ym for _i in range(len(test_y))]
    else:
        print ("model parameter not recognized")

    test_prob = np.reshape(np.array(test_prob), (-1))
    predsbool = np.reshape(np.array(test_pred).astype(bool), (-1))
    obsvsbool = np.reshape(np.array(test_y).astype(bool), (-1))

    bacc3 = balancedacc(obsvsbool, test_prob, 0.3)
    bacc5 = balancedacc(obsvsbool, test_prob, 0.5)
    bacc7 = balancedacc(obsvsbool, test_prob, 0.7)
    mbacc = maxbacc(obsvsbool, test_prob)
    fpr, tpr, thresholds = sklearn.metrics.roc_curve(obsvsbool, test_prob)
    print ("")
    print ("for model ", model)
    sauc = subjauc(subjs, obsvsbool, test_prob)
    auc = sklearn.metrics.auc(fpr, tpr)
    ll = sklearn.metrics.log_loss(np.squeeze(obsvsbool), np.squeeze(test_prob)) / float(len(test_prob))
    print ("balanced accuracy is : ", round(bacc5,4))
    print ("auc is : ", round(auc,4))
    print ("log loss is : ", '%.4E' % Decimal(ll))
    return [bacc3, bacc5, bacc7, mbacc, fpr, tpr, auc, sauc, ll]

models =  ["topfeatures"]
NFEATSs = [len(featrank)]

filename1 = '_normed_standardized_Actual_EV0.5' # to load y
filename2 = '_normed_standardized_Actual_EV0.5_interactions' # to load x

traintype = 'random'
testtype = 'random'

if traintype == "1block":
    trainsuffix = "_1block"
elif traintype == "random":
    trainsuffix = "_random"

if testtype == "3block":
    testsuffix = "_3block"
elif testtype == "1block":
    testsuffix = "_1block"
elif testtype == "random":
    testsuffix = "_random"


with open('results.csv', 'w') as writeFile:
    writer = csv.writer(writeFile)

    LAMBDA = 0.1

    # all subjects
    writer.writerow(["all test subjects"])
    writer.writerow([LAMBDA])
    writer.writerow(["model", "number of features", "bacc3","bacc5","bacc7", "mbacc", "auc", "sauc", "ll"])
    for model, NFEATS, color in zip(models, NFEATSs, colors):
        [bacc3, bacc5, bacc7, mbacc, fpr, tpr, auc, sauc, ll] = runmodel(model, traintype, testtype, NFEATS)
        writer.writerow([model, str(NFEATS), round(bacc3,4), round(bacc5,4),
            round(bacc7,4), round(mbacc,4),
            round(auc,4),round(sauc,4),'%.4E' % Decimal(ll)])

    LAMBDA = 1.0

    # all subjects
    writer.writerow(["all test subjects"])
    writer.writerow([LAMBDA])
    writer.writerow(["model", "number of features", "bacc3","bacc5","bacc7", "mbacc", "auc", "sauc", "ll"])
    for model, NFEATS, color in zip(models, NFEATSs, colors):
        [bacc3, bacc5, bacc7, mbacc, fpr, tpr, auc, sauc, ll] = runmodel(model, traintype, testtype, NFEATS)
        writer.writerow([model, str(NFEATS), round(bacc3,4), round(bacc5,4),
            round(bacc7,4), round(mbacc,4),
            round(auc,4),round(sauc,4),'%.4E' % Decimal(ll)])

    LAMBDA = 10.0

    # all subjects
    writer.writerow(["all test subjects"])
    writer.writerow([LAMBDA])
    writer.writerow(["model", "number of features", "bacc3","bacc5","bacc7", "mbacc", "auc", "sauc", "ll"])
    for model, NFEATS, color in zip(models, NFEATSs, colors):
        [bacc3, bacc5, bacc7, mbacc, fpr, tpr, auc, sauc, ll] = runmodel(model, traintype, testtype, NFEATS)
        writer.writerow([model, str(NFEATS), round(bacc3,4), round(bacc5,4),
            round(bacc7,4), round(mbacc,4),
            round(auc,4),round(sauc,4),'%.4E' % Decimal(ll)])
writeFile.close()

[Back to Table of Contents](#toc)

## Estimate Weights of a Model Trained on Entire Data Set

Output: 
* npy files with weights of logistic regression

In [ ]:
ET = "random"

filename = '_normed_standardized_Actual_EV'
gamma = 0.5
suffix = "_"+ET

LAMBDAs = [0.1, 1.0, 10.0]

x = np.genfromtxt("x" + suffix + filename + str(gamma) + "_interactions.csv", delimiter=",")
y = np.genfromtxt("y" + suffix + filename + str(gamma) + ".csv", delimiter=",")


for LAMBDA in LAMBDAs:
    xs = x[1:,1:]
    ys = y[1:,1:]
    print (xs.shape)
    print (ys.shape)

    clf = LogisticRegression(penalty = 'l1', C = 1/LAMBDA, solver='liblinear')
    clf.fit(xs, ys.ravel())
    w = clf.coef_.ravel()

    np.save(ET +"_lambda" + str(LAMBDA) +  "_weights", w)
    print (ET +"_lambda" + str(LAMBDA) +  "_weights")

[Back to Table of Contents](#toc)

## Visualize Weights of a Model Trained on Entire Data Set

Output: 
* csv file of model weights

In [ ]:
LAMBDA = 1.0


ET1 = "random"

wnames = np.load('x_random_normed_standardized_Actual_EV0.5_interactions_names.npy')
w1 = np.load(ET1 +"_lambda" + str(LAMBDA) + "_weights.npy")

wm = [np.absolute(i) for i in w1]

w1 = [x for _,x in sorted(zip(wm,w1), reverse=True)]
wnames = [x for _,x in sorted(zip(wm,wnames), reverse=True)]

with open('model_weights_'+ET1+'.csv', 'w') as writeFile:
    writer = csv.writer(writeFile)
    for i in range(len(wnames)):
        writer.writerow([wnames[i], w1[i]])

[Back to Table of Contents](#toc)

## Compare Weights of a Model Trained on Entire Data Set

Output: 
* heatmap with weights of logistic regression

In [ ]:
LAMBDA = 10.0


ET1 = "random"
ET2 = "random"

wnames = np.load('x_random_normed_standardized_Actual_EV0.5_interactions_names.npy')
w1 = np.load(ET1 +"_lambda" + str(LAMBDA) + "_weights.npy")
w2 = np.load(ET2 +"_lambda" + str(LAMBDA) + "_weights.npy")



wm = [np.maximum(np.absolute(i),np.absolute(j)) for i,j in zip(w1,w2)]


w1 = [x for _,x in sorted(zip(wm,w1), reverse=True)]
w2 = [x for _,x in sorted(zip(wm,w2), reverse=True)]
wnames = [x for _,x in sorted(zip(wm,wnames), reverse=True)]



NFEATS = 10

with open('names.csv', 'w') as writeFile:
    writer = csv.writer(writeFile)
    for i in range(NFEATS):
        writer.writerow([wnames[i]])


matrix = np.transpose(np.array([w1[:NFEATS], w2[:NFEATS]]))

cmap = sns.diverging_palette(250, 10, as_cmap=True)
ax = sns.heatmap(matrix, center=0, annot=True, fmt=".1f",
                yticklabels=wnames[:NFEATS], xticklabels=[ET1, ET2], cmap=cmap)
plt.show()

[Back to Table of Contents](#toc)

# Neural Network Modeling and Interpretation

## Estimate LOOCV accuracy

Output: 
* csv file of results

In [ ]:
UNITS = 5
SUFFIX = "nn"
training_epochs = 3000
LAYERS = 2
REG = "l1"
NOISE = "adversarial"
adv = "sign" # can include "nosign"

LAMBDAs = [1e-1]
epsilons = [0.0]
nSEEDS = 3


def run(NOISE, adv, LAMBDA, epsilon, REG, SEED):
    tf.set_random_seed(SEED)

    x = np.genfromtxt('x_random_normed_standardized_Actual_EV0.5_stdz.csv', delimiter=",")
    y = np.genfromtxt('y_random_normed_standardized_Actual_EV0.5.csv', delimiter=",")

    sf = np.genfromtxt('gps_random.csv', delimiter=",", usecols=((0,1)))  # subject file
    ORIGsids = np.array(sf[1:,1]) # subject ids
    n = len(ORIGsids)

    if SEED == 0:
        sids = ORIGsids
    else:
        np.random.seed(SEED) ; sids = np.random.choice(ORIGsids, len(ORIGsids))

    # newer version

    X = tf.placeholder(tf.float32, shape = [None,x.shape[1]-1])
    N = tf.placeholder(tf.float32, shape = [None,x.shape[1]-1])
    Ga = tf.placeholder(tf.float32, shape = [None,x.shape[1]-1])
    Y = tf.placeholder(tf.float32, shape = [None, 1])
    if REG == "l2":
        l = tf.contrib.layers.l2_regularizer(scale=LAMBDA)
    elif REG == "l1":
        l = tf.contrib.layers.l1_regularizer(scale=LAMBDA)
    else:
        print("parameter 'REG' not recognized")
    if LAYERS == 2:
        H1 = tf.contrib.layers.fully_connected(X,UNITS,activation_fn=tf.nn.sigmoid,weights_regularizer=l)
        H2 = tf.contrib.layers.fully_connected(H1,UNITS,activation_fn=tf.nn.sigmoid,weights_regularizer=l)
        L = tf.contrib.layers.fully_connected(H2,1,activation_fn=None,weights_regularizer=l)
    elif LAYERS == 1:
        H1 = tf.contrib.layers.fully_connected(X,UNITS,activation_fn=tf.nn.sigmoid,weights_regularizer=l)
        L = tf.contrib.layers.fully_connected(H1,1,activation_fn=None,weights_regularizer=l)
    else:
        print ("LAYERS parameter not recognized")
    l_loss = tf.losses.get_regularization_loss()
    P = tf.nn.sigmoid(L)
    cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y,logits=L)) # cross entropy
    if adv == "sign":
        A = tf.stop_gradient(tf.sign(tf.gradients(cost,X)))
    elif adv == "nosign":
        A = tf.stop_gradient(tf.gradients(cost,X))
    else:
        print ("not recognized value of adv")
    XA = tf.stop_gradient(X + N * A)
    XGa = tf.stop_gradient(X + N * Ga)
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost+l_loss)
    G = tf.stop_gradient(tf.gradients(L,X))
    H = tf.stop_gradient(tf.reduce_sum(tf.squeeze(tf.hessians(L,X)),axis=2))

    init = tf.global_variables_initializer()
    saver = tf.train.Saver()


    final_test_costs = []
    obsvs = []
    preds = []
    grads = []
    for fold in range(n):
        [test_x, test_y] = subjXY(sids[fold], x, y)
        [train_x, train_y] = notsubjXY(sids[fold], x, y)
        test_y = np.reshape(test_y, (-1,1))
        train_y = np.reshape(train_y, (-1,1))

        o = test_y
        with tf.Session() as sess:
            sess.run(init)

            # try new version from nn_boot_hessians.py
            for epoch in range(training_epochs):
                n = np.random.uniform(low=0.0, high=epsilon, size=train_x.shape)
                xa = np.squeeze(sess.run(XA, feed_dict={X : train_x, Y: 1.0 - train_y, N : n}))
                if epoch != training_epochs - 1:
                    if NOISE == "adversarial":
                        _, c = sess.run([optimizer, cost], feed_dict={X: np.concatenate([train_x, xa], axis=0), Y: np.concatenate([train_y, train_y], axis=0)})
                    elif NOISE == "gaussian":
                        ga = np.random.normal(size = train_x.shape)
                        xga = np.squeeze(sess.run(XGa, feed_dict={X : train_x, Ga: ga, N : n}))
                        _, c= sess.run([optimizer, cost], feed_dict={X: np.concatenate([train_x, xga], axis=0), Y: np.concatenate([train_y, train_y], axis=0)})
                    else:
                        print ("encountered error")
                else:
                    if NOISE == "adversarial":
                        _, c, g = sess.run([optimizer, cost, G], feed_dict={X: np.concatenate([train_x, xa], axis=0), Y: np.concatenate([train_y, train_y], axis=0)})
                    elif NOISE == "gaussian":
                        ga = np.random.normal(size = xs.shape)
                        xga = np.squeeze(sess.run(XGa, feed_dict={X : train_x, Ga: ga, N : n}))
                        _, c, g = sess.run([optimizer, cost, G], feed_dict={X: np.concatenate([train_x, xga], axis=0), Y: np.concatenate([train_y, train_y], axis=0)})
                    else:
                        print ("encountered error")
                print("Epoch: {} - train cost: {}".format(epoch, c))
            p = sess.run(P, feed_dict={X: test_x})
            print (p)

            saver.save(sess, "./model"+str(fold+1)+"_"+SUFFIX+"_lambda"+str(LAMBDA)+".ckpt") # checkpoint
        preds.append(p)
        obsvs.append(o)
        grads.append(g)
    np.save(SUFFIX+"_SEED"+str(SEED)+"_layers"+str(LAYERS)+"_lambda"+str(LAMBDA)+"_adv"+str(adv)+"_reg"+str(REG)+"_units"+str(UNITS)+"_pred.npy", preds)
    np.save(SUFFIX+"_SEED"+str(SEED)+"_layers"+str(LAYERS)+"_lambda"+str(LAMBDA)+"_adv"+str(adv)+"_reg"+str(REG)+"_units"+str(UNITS)+"_obsv.npy", obsvs)
    np.save(SUFFIX+"_SEED"+str(SEED)+"_layers"+str(LAYERS)+"_lambda"+str(LAMBDA)+"_adv"+str(adv)+"_reg"+str(REG)+"_units"+str(UNITS)+"_grads.npy", grads)
    np.save(SUFFIX+"_SEED"+str(SEED)+"_layers"+str(LAYERS)+"_lambda"+str(LAMBDA)+"_adv"+str(adv)+"_reg"+str(REG)+"_units"+str(UNITS)+"_testcosts.npy", final_test_costs)


    f = lambda x : x > 0.5
    predsbool = np.array(map(f, preds))
    totaln = len(predsbool) * len(predsbool[0])
    preds = np.reshape(np.array(preds), [totaln])
    predsbool = np.reshape(predsbool, [totaln])
    obsvsbool = np.squeeze(np.array(obsvs).astype(bool))
    obsvsbool = np.reshape(obsvsbool, [totaln])

    bacc = balancedacc(obsvsbool, preds, 0.5)

    acc = sklearn.metrics.accuracy_score(obsvsbool, predsbool)
    fpr, tpr, thresholds = sklearn.metrics.roc_curve(obsvsbool, predsbool)
    auc = sklearn.metrics.auc(fpr, tpr)
    print (SEED, LAMBDA, epsilon)
    print (sids)
    print ("balanced accuracy is : ", round(bacc,4))
    print ("auc is : ", auc)

    return [round(bacc,4), auc]#, ll]

# just in case it breaks somewhere
with open('nn_folds_results.csv', 'w') as f:
    w = csv.writer(f)
    results = []
    for SEED in range(nSEEDS):
        for epsilon in epsilons:
            for LAMBDA in LAMBDAs:
                print (epsilon, LAMBDA, SEED)
                result = run(NOISE, adv, LAMBDA, epsilon, REG, SEED)
                result.extend(["epsilon : " + str(epsilon), "lambda : "+ str(LAMBDA), "SEED : " + str(SEED)])
                w.writerow(result)
                results.append(result)
    f.close()

# all the results
with open('nn_folds_results_all.csv', 'w') as f:
    fw = csv.writer(f)
    fw.writerows(results)
    f.close()

[Back to Table of Contents](#toc)

## Generate Hessian Values
Output
* npy files with hessian variables

In [ ]:
x = np.genfromtxt('x_random_normed_standardized_Actual_EV0.5_stdz.csv', delimiter=",")
y = np.genfromtxt('y_random_normed_standardized_Actual_EV0.5.csv', delimiter=",")

# get column names
xtest = np.genfromtxt('x_random_normed_standardized_Actual_EV0.5_stdz.csv', delimiter=",", names=True)
xcolnames = xtest.dtype.names
fnames = xcolnames[1:]
nFEATS = len(fnames)
np.save("featnames", fnames)


sf = np.genfromtxt('gps_random.csv', delimiter=",", usecols=((0,1))) # subject file
ORIGsids = np.array(sf[1:,1]) # subject ids

SUFFIX = "nnboot"
UNITS = 5
training_epochs = 3000
LAYERS = 2
REGs = ["l1"]
noises = ["adversarial"]
advs = ["sign"] # can include "nosign"
LAMBDAs = [1e-4]
epsilons = [1e-1] # adversarial noise
nSEEDS = 1


epsilonSGs = [0.2] # smooth grad

nSMOOTHGRAD = 50
nSUBJ =  len(ORIGsids)
nOBSV = y.shape[0] - 1
nTRIALS = nOBSV / nSUBJ

def check_symmetric(a, rtol=1e-05, atol=1e-08):
    return np.allclose(a, a.T, rtol=rtol, atol=atol)

def runseeds(NOISE, adv, LAMBDA, epsilon, REG, epsilonSG):
    hesss = np.zeros(shape=[nSEEDS*nSUBJ*nTRIALS,nFEATS,nFEATS])
    for SEED in range(1,nSEEDS + 1):

        tf.set_random_seed(SEED)

        if SEED == 0:
            sids = ORIGsids
        else:
            np.random.seed(SEED) ; sids = np.random.choice(ORIGsids, len(ORIGsids))

        xs = np.zeros(shape=[x.shape[0]-1, x.shape[1]-1])
        ys = np.zeros(shape=[y.shape[0]-1, 1])
        xbool = np.zeros(shape=y.shape[0]-1)

        counter = 0
        for i in range(len(sids)):
            [sx, sy, size] = subjXY(sids[i], x, y)
            xs[range(counter, counter+size), :] = sx
            ys[range(counter, counter+size)] = sy
            counter = counter + size

        X = tf.placeholder(tf.float32, shape = [None,xs.shape[1]])
        N = tf.placeholder(tf.float32, shape = [None,xs.shape[1]])
        Ga = tf.placeholder(tf.float32, shape = [None,xs.shape[1]])
        Y = tf.placeholder(tf.float32, shape = [None, 1])
        if REG == "l2":
            l = tf.contrib.layers.l2_regularizer(scale=LAMBDA)
        elif REG == "l1":
            l = tf.contrib.layers.l1_regularizer(scale=LAMBDA)
        else:
            print("parameter 'REG' not recognized")
        if LAYERS == 2:
            H1 = tf.contrib.layers.fully_connected(X,UNITS,activation_fn=tf.nn.sigmoid,weights_regularizer=l)
            H2 = tf.contrib.layers.fully_connected(H1,UNITS,activation_fn=tf.nn.sigmoid,weights_regularizer=l)
            L = tf.contrib.layers.fully_connected(H2,1,activation_fn=None,weights_regularizer=l)
        elif LAYERS == 1:
            H1 = tf.contrib.layers.fully_connected(X,UNITS,activation_fn=tf.nn.sigmoid,weights_regularizer=l)
            L = tf.contrib.layers.fully_connected(H1,1,activation_fn=None,weights_regularizer=l)
        else:
            print ("LAYERS parameter not recognized")
        l_loss = tf.losses.get_regularization_loss()
        P = tf.nn.sigmoid(L)
        cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y,logits=L)) # cross entropy
        if adv == "sign":
            A = tf.stop_gradient(tf.sign(tf.gradients(cost,X)))
        elif adv == "nosign":
            A = tf.stop_gradient(tf.gradients(cost,X))
        else:
            print ("not recognized value of adv")
        XA = tf.stop_gradient(X + N * A)
        XGa = tf.stop_gradient(X + N * Ga)
        optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost+l_loss)
        G = tf.stop_gradient(tf.gradients(L,X))
        H = tf.stop_gradient(tf.reduce_sum(tf.squeeze(tf.hessians(L,X)),axis=2))

        init = tf.global_variables_initializer()
        saver = tf.train.Saver()

        with tf.Session() as sess:
            sess.run(init)
            for epoch in range(training_epochs):
                n = np.random.uniform(low=0.0, high=epsilon, size=xs.shape)
                xa = np.squeeze(sess.run(XA, feed_dict={X : xs, Y: 1.0 - ys, N : n}))
                if epoch != training_epochs - 1:
                    if NOISE == "adversarial":
                        _, c = sess.run([optimizer, cost], feed_dict={X: np.concatenate([xs, xa], axis=0), Y: np.concatenate([ys, ys], axis=0)})
                    elif NOISE == "gaussian":
                        ga = np.random.normal(size = xs.shape)
                        xga = np.squeeze(sess.run(XGa, feed_dict={X : xs, Ga: ga, N : n}))
                        _, c= sess.run([optimizer, cost], feed_dict={X: np.concatenate([xs, xga], axis=0), Y: np.concatenate([ys, ys], axis=0)})
                    else:
                        print ("encountered error")
                else:
                    if NOISE == "adversarial":
                        _, c, g, h = sess.run([optimizer, cost, G, H], feed_dict={X: np.concatenate([xs, xa], axis=0), Y: np.concatenate([ys, ys], axis=0)})
                        print ("is symmetric? : ", check_symmetric(h[0,:,:]))
                        print ("is symmetric? : ", check_symmetric(h[211,:,:]))
                        print ("is symmetric? : ", check_symmetric(h[562,:,:]))
                        print ("is symmetric? : ", check_symmetric(h[736,:,:]))
                    elif NOISE == "gaussian":
                        ga = np.random.normal(size = xs.shape)
                        xga = np.squeeze(sess.run(XGa, feed_dict={X : xs, Ga: ga, N : n}))
                        _, c, g, h = sess.run([optimizer, cost, G, H], feed_dict={X: np.concatenate([xs, xga], axis=0), Y: np.concatenate([ys, ys], axis=0)})
                    else:
                        print ("encountered error")
            _save_path = saver.save(sess, "./model_temp" +"_SEED"+str(SEED)+"_layers"+str(LAYERS)+"_"+REG+"_lambda"+str(LAMBDA)+"_epsilon"+str(epsilon)+"_epsilonSG"+str(epsilonSG)+"_noise"+NOISE+"_sign"+adv+"_units"+str(UNITS)+ ".ckpt")



            saver.restore(sess, "./model_temp" +"_SEED"+str(SEED)+"_layers"+str(LAYERS)+"_"+REG+"_lambda"+str(LAMBDA)+"_epsilon"+str(epsilon)+"_epsilonSG"+str(epsilonSG)+"_noise"+NOISE+"_sign"+adv+"_units"+str(UNITS)+ ".ckpt")

            h = sess.run(H, feed_dict={X: xs, Y: ys})

            # regular hessian
            h = np.squeeze(h)
            print ("hess non-zero entries : ", np.not_equal(h,0.0).sum())
            print ("hess shape is : ", h.shape)
            hesss[(SEED-1)*nSUBJ*nTRIALS: (SEED)*nSUBJ*nTRIALS,:,:] = h[range(nSUBJ*nTRIALS), :, :]
            if NOISE == "adversarial":
                np.save(SUFFIX+"_SEED"+str(SEED)+"_layers"+str(LAYERS)+"_"+REG+"_lambda"+str(LAMBDA)+"_epsilon"+str(epsilon)+"_epsilonSG"+str(epsilonSG)+"_noise"+NOISE+"_sign"+adv+"_units"+str(UNITS)+"_hesss.npy", np.squeeze(h))
            elif NOISE == "gaussian":
                np.save(SUFFIX+"_SEED"+str(SEED)+"_layers"+str(LAYERS)+"_"+REG+"_lambda"+str(LAMBDA)+"_epsilon"+str(epsilon)+"_epsilonSG"+str(epsilonSG)+"_noise"+NOISE+"_units"+str(UNITS)+"_hesss.npy", np.squeeze(h))
            else:
                print ("encountered error")

            # smooth grad hessians

            # smooth grad
            hess_sums = np.zeros(shape = np.squeeze(h).shape)
            for i in range(nSMOOTHGRAD):
                np.random.seed(i); n = np.random.uniform(low=0.0, high=epsilonSG, size=xs.shape)
                np.random.seed(i); ga = np.random.normal(size = xs.shape)
                xga = np.squeeze(sess.run(XGa, feed_dict={X : xs, Ga: ga, N : n}))
                h = sess.run(H, feed_dict={X: xga, Y: ys})
                hess_sums = hess_sums + np.squeeze(h)

            if NOISE == "adversarial":
                np.save(SUFFIX+"_SEED"+str(SEED)+"_"+REG+"_lambda"+str(LAMBDA)+"_epsilon"+str(epsilon)+"_epsilonSG"+str(epsilonSG)+"_noise"+NOISE+"_sign"+adv+"_units"+str(UNITS)+"_points"+ str(nSMOOTHGRAD) + "_hesss_sums.npy", hess_sums)
            elif NOISE == "gaussian":
                np.save(SUFFIX+"_SEED"+str(SEED)+"_"+REG+"_lambda"+str(LAMBDA)+"_epsilon"+str(epsilon)+"_epsilonSG"+str(epsilonSG)+"_noise"+NOISE+"_units"+str(UNITS)+"_points"+ str(nSMOOTHGRAD) +"_hesss_sums.npy", hess_sums)
            else:
                print ("encountered error")

    if NOISE == "adversarial":
        np.save(SUFFIX+"_nSEED"+str(nSEEDS)+"_layers"+str(LAYERS)+"_"+REG+"_lambda"+str(LAMBDA)+"_epsilon"+str(epsilon)+"_epsilonSG"+str(epsilonSG)+"_noise"+NOISE+"_sign"+adv+"_units"+str(UNITS)+"_hesss.npy", hesss)
    elif NOISE == "gaussian":
        np.save(SUFFIX+"_nSEED"+str(nSEEDS)+"_layers"+str(LAYERS)+"_"+REG+"_lambda"+str(LAMBDA)+"_epsilon"+str(epsilon)+"_epsilonSG"+str(epsilonSG)+"_noise"+NOISE+"_units"+str(UNITS)+"_hesss.npy", hesss)
    else:
        print ("encountered error")

for NOISE in noises:
    for REG in REGs:
        for adv in advs:
            for LAMBDA in LAMBDAs:
                for epsilon in epsilons:
                    for epsilonSG in epsilonSGs:
                        print NOISE, adv, LAMBDA, epsilon, epsilonSG
                        runseeds(NOISE, adv, LAMBDA, epsilon, REG, epsilonSG)

[Back to Table of Contents](#toc)

## Visualize Hessian Matrices

Output: 
* heatmaps of prevalence, positive ratios, negative ratios and average hessian values

In [ ]:
plt.rcParams.update({'font.size': 8})

In [ ]:
def subprocess_cmd(command):
    process = subprocess.Popen(command.split(), stdout=subprocess.PIPE)
    output, error = process.communicate()

# ratio of positives in list l
def posrate(l):
    b = len(l)
    a = 0
    for e in l:
        if e > 0.0:
            a = a + 1
    return float(a) / float(b)

# ratio of negatives in list l
def negrate(l):
    b = len(l)
    a = 0
    for e in l:
        if e < 0.0:
            a = a + 1
    return float(a) / float(b)

def visSG(NOISE, adv, LAMBDA, epsilon, REG, SEED):
    # smooth grads

    fn = SUFFIX+"_SEED"+str(SEED)+"_"+REG+"_lambda"+str(LAMBDA)+"_epsilon"+str(epsilon)+"_epsilonSG"+str(epsilonSG)+"_noise"+NOISE+"_sign"+adv+"_units"+str(UNITS)+"_points"+ str(nSMOOTHGRAD) + "_hesss_sums.npy"
    hessssums = np.load(fn)
    hesss = hessssums / nSMOOTHGRAD
    print (hesss.shape)
    for ind in range(hesss.shape[0]):
        hi = hesss[ind,:,:]

    means = hesss.mean(axis=0)
    stds = hesss.std(axis=0)

    poss = np.zeros(shape = means.shape)
    negs = np.zeros(shape = means.shape)

    for i in range(nFEATS):
        for j in range(nFEATS):
            v = hesss[:,i,j]
            poss[i,j] = posrate(v)
            negs[i,j] = negrate(v)

    print ("are the poss symmetric? : ", check_symmetric(poss))
    print ("are the negs symmetric? : ", check_symmetric(negs))

    with open('hessian_smoothgrad_results.csv', 'w') as f:
        fw = csv.writer(f)
        fw.writerow(['means'])
        fw.writerow(fnames)
        fw.writerows(means)
        fw.writerow(['stds'])
        fw.writerow(fnames)
        fw.writerows(stds)
        fw.writerow(['poss'])
        fw.writerow(fnames)
        fw.writerows(poss)
        fw.writerow(['negs'])
        fw.writerow(fnames)
        fw.writerows(negs)
        f.close()

    return [means, stds, poss, negs]

def visREG(NOISE, adv, LAMBDA, epsilon, REG, SEED):
    fn = SUFFIX+"_SEED"+str(SEED)+"_layers"+str(LAYERS)+"_"+REG+"_lambda"+str(LAMBDA)+"_epsilon"+str(epsilon)+"_epsilonSG"+str(epsilonSG)+"_noise"+NOISE+"_sign"+adv+"_units"+str(UNITS)+"_hesss"
    hesss = np.load(fn+".npy")

    means = hesss.mean(axis=0)
    stds = hesss.std(axis=0)

    poss = np.zeros(shape = means.shape)
    negs = np.zeros(shape = means.shape)

    for i in range(nFEATS):
        for j in range(nFEATS):
            poss[i,j] = posrate(hesss[i,j])
            negs[i,j] = negrate(hesss[i,j])


    with open('hessian_results.csv', 'w') as f:
        fw = csv.writer(f)
        fw.writerow(['means'])
        fw.writerow(fnames)
        fw.writerows(means)
        fw.writerow(['stds'])
        fw.writerow(fnames)
        fw.writerows(stds)
        fw.writerow(['poss'])
        fw.writerow(fnames)
        fw.writerows(poss)
        fw.writerow(['negs'])
        fw.writerow(fnames)
        fw.writerows(negs)
        f.close()
        f.close()

    return [means, stds, poss, negs]

for NOISE in noises:
    for REG in REGs:
        for adv in advs:
            for LAMBDA in LAMBDAs:
                for epsilon in epsilons:
                    for epsilonSG in epsilonSGs:
                        # positive ratio
                        psgs = np.zeros(shape=(nSEEDS, nFEATS, nFEATS))
                        # negative ratio
                        nsgs = np.zeros(shape=(nSEEDS, nFEATS, nFEATS))
                        # positive ratio minus negative ratio in smooth grads
                        pmnsgs = np.zeros(shape=(nSEEDS, nFEATS, nFEATS))
                        # means
                        msgs = np.zeros(shape=(nSEEDS, nFEATS, nFEATS))
                        for SEED in range(1,nSEEDS+1):
                            print (NOISE, adv, LAMBDA, epsilon, SEED)
                            msg, ssg, psg, nsg = visSG(NOISE, adv, LAMBDA, epsilon, REG, SEED)
                            print ("is psg symmetric? : ", check_symmetric(psg))
                            print ("is nsg symmetric? : ", check_symmetric(nsg))
                            psgs[SEED - 1, :, :] = psg
                            nsgs[SEED - 1, :, :] = nsg
                            pmnsgs[SEED - 1, :, :] = psg - nsg
                            msgs[SEED - 1, :, :] = msg


                        cmap = sns.diverging_palette(250, 10, as_cmap=True)
                        ax = sns.heatmap(msgs.mean(axis=0), center=0, annot=True, fmt=".4f",
                                         yticklabels=fnames , cmap=cmap) # xticklabels=fnames,
                        ax.set_title("hessian smoothgrad average Hessian value across " +str(nSEEDS)+ " resamplings")
                        plt.show()

                        cmap = sns.diverging_palette(250, 10, as_cmap=True)
                        ax = sns.heatmap(pmnsgs.mean(axis=0), center=0, annot=True, fmt=".4f",
                                         yticklabels=fnames , cmap=cmap) # xticklabels=fnames,
                        ax.set_title("hessian smoothgrad expected prevalence across " +str(nSEEDS)+ " resamplings")
                        plt.show()

                        cmap = sns.diverging_palette(250, 10, as_cmap=True)
                        ax = sns.heatmap(psgs.mean(axis=0), center = 0.5, annot=True, fmt=".3f",
                                         yticklabels=fnames , cmap=cmap) # xticklabels=fnames,
                        ax.set_title("hessian smoothgrad expected positive ratio across " +str(nSEEDS)+ " resamplings")
                        plt.show()

                        cmap = sns.diverging_palette(250, 10, as_cmap=True)
                        ax = sns.heatmap(nsgs.mean(axis=0), center = 0.5, annot=True, fmt=".3f",
                                         yticklabels=fnames , cmap=cmap) # xticklabels=fnames,
                        ax.set_title("hessian smoothgrad expected negative ratio across " +str(nSEEDS)+ " resamplings")
                        plt.show()

[Back to Table of Contents](#toc)